In [1]:
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

tf.random.set_seed(42)
print("GPUs:", tf.config.list_physical_devices('GPU'))

2025-09-14 05:31:06.917820: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-14 05:31:07.557079: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-14 05:31:09.828724: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
print(tf.__version__)

2.20.0


In [3]:
# import tensorflow as tf
# from typing import List, Dict, Tuple

# def tokenize_and_build_vocabulary_tf(
#     file_path_list: List[str] = [r"/home/akshat/GPT_from_scratch/text_data/pg76702.txt"],
#     existing_vocab: Dict[str, int] = None # type: ignore
# ) -> Tuple[tf.lookup.StaticHashTable, tf.lookup.StaticHashTable]:
#     """
#     Build a character-level vocabulary from text files and return TensorFlow lookup tables.
    
#     Returns:
#         token_to_id_table: tf.lookup.StaticHashTable mapping char -> int
#         id_to_token_table: tf.lookup.StaticHashTable mapping int -> char
#     """
#     if existing_vocab is None:
#         existing_vocab = {}

#     vocab_set = set(existing_vocab.keys())

#     # Collect characters from all files
#     for file_name in file_path_list:
#         with open(file_name, encoding="utf-8") as f:
#             text = f.read()
#             vocab_set.update(text)

#     # Sort for consistency
#     sorted_tokens = sorted(vocab_set)

#     # Assign IDs (keep existing IDs if possible)
#     token_to_id = {token: i for i, token in enumerate(sorted_tokens)}
#     id_to_token = {i: token for token, i in token_to_id.items()}

#     # Convert dicts to tensors
#     token_keys = tf.constant(list(token_to_id.keys()))
#     token_values = tf.constant(list(token_to_id.values()), dtype=tf.int32)

#     id_keys = tf.constant(list(id_to_token.keys()), dtype=tf.int32)
#     id_values = tf.constant(list(id_to_token.values()))

#     # Create TensorFlow lookup tables
#     token_to_id_table = tf.lookup.StaticHashTable(
#         initializer=tf.lookup.KeyValueTensorInitializer(token_keys, token_values),
#         default_value=-1  # unknown token
#     )
#     id_to_token_table = tf.lookup.StaticHashTable(
#         initializer=tf.lookup.KeyValueTensorInitializer(id_keys, id_values),
#         default_value=""  # unknown ID
#     )

#     return token_to_id_table, id_to_token_table


In [4]:
# import os
# from typing import List, Dict, Tuple
# import tensorflow as tf

# def tokenize_and_build_vocabulary_tf(
#     file_path_list: List[str],
#     existing_vocab: Dict[str, int] | None = None
# ) -> Tuple[tf.lookup.StaticHashTable, tf.lookup.StaticHashTable]:
#     """
#     Build a character-level vocabulary from text files and return TF lookup tables:
#       token_to_id: char -> int
#       id_to_token: int -> char
#     """
#     if isinstance(file_path_list, (str, bytes)):
#         file_path_list = [file_path_list] # type: ignore
#     if existing_vocab is None:
#         existing_vocab = {}

#     vocab_set = set(existing_vocab.keys())
#     for file_name in file_path_list:
#         if os.path.isdir(file_name):
#             raise IsADirectoryError(f"Expected file path, got directory: {file_name}")
#         if not os.path.isfile(file_name):
#             raise FileNotFoundError(f"File not found: {file_name}")
#         with open(file_name, encoding="utf-8") as f:
#             text = f.read()
#             vocab_set.update(text)

#     sorted_tokens = sorted(vocab_set)
#     token_to_id = {tok: i for i, tok in enumerate(sorted_tokens)}
#     id_to_token = {i: tok for tok, i in token_to_id.items()}

#     token_keys = tf.constant(list(token_to_id.keys()), dtype=tf.string)
#     token_vals = tf.constant(list(token_to_id.values()), dtype=tf.int32)

#     id_keys = tf.constant(list(id_to_token.keys()), dtype=tf.int32)
#     id_vals = tf.constant(list(id_to_token.values()), dtype=tf.string)

#     token_to_id_table = tf.lookup.StaticHashTable(
#         initializer=tf.lookup.KeyValueTensorInitializer(token_keys, token_vals),
#         default_value=-1
#     )
#     id_to_token_table = tf.lookup.StaticHashTable(
#         initializer=tf.lookup.KeyValueTensorInitializer(id_keys, id_vals),
#         default_value=tf.constant("", dtype=tf.string)
#     )
#     return token_to_id_table, id_to_token_table


In [5]:
@tf.function
@keras.saving.register_keras_serializable()
def prepare_sinusoidal_lookup_table(EMBEDDING_SIZE: int = 128, max_seq_len: int = 512):
    """
    Builds a sinusoidal positional encoding lookup table.
    
    Args:
      EMBEDDING_SIZE: dimensionality of each position encoding vector (must be even).
      max_seq_len: maximum sequence length (number of positions).
    
    Returns:
      lookup_table: a tf array of shape (max_seq_len, EMBEDDING_SIZE)
                    where row p gives the positional encoding for position p.
    """
    # Initialize the table
    lookup_table = np.zeros((max_seq_len, EMBEDDING_SIZE), dtype=np.float32)
    
    # Compute the angle rates for each dimension
    # angle_rates[k] = 1 / (10000^(2*(k//2) / EMBEDDING_SIZE))
    dims = np.arange(EMBEDDING_SIZE)[np.newaxis, :]   # shape (1, EMBEDDING_SIZE)
    positions = np.arange(max_seq_len)[:, np.newaxis] # shape (max_seq_len, 1)
    angle_rates = 1 / np.power(10000, (2 * (dims // 2)) / EMBEDDING_SIZE)
    
    # Compute the angle for each position and dimension: position * angle_rate
    angle_rads = positions * angle_rates  # shape (max_seq_len, EMBEDDING_SIZE)
    
    # Apply sin to even indices (0,2,4,...) and cos to odd indices (1,3,5,...)
    lookup_table[:, 0::2] = np.sin(angle_rads[:, 0::2])
    lookup_table[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    return tf.constant(lookup_table)

In [6]:
# import tensorflow as tf

# def tokenize_and_build_token_id(token_to_id_table: tf.lookup.StaticHashTable,
#                                 text_batch: tf.Tensor,
#                                 pad_value: int = 0):
#     """
#     Tokenize a batch of strings character by character, pad sequences,
#     and return attention masks.

#     Args:
#         token_to_id_table: TF lookup table mapping char -> int
#         text_batch: tf.Tensor of shape [batch_size], dtype=tf.string
#         pad_value: int, ID to use for padding

#     Returns:
#         token_ids: tf.Tensor [batch_size, max_seq_len]
#         attention_mask: tf.Tensor [batch_size, max_seq_len]
#     """
#     token_ids_list = []

#     for text in text_batch.numpy():  # type: ignore
#         # Convert bytes to TF string

#         # Split into characters
#         char_tensor = tf.strings.bytes_split(text)

#         # Lookup token IDs
#         token_ids = token_to_id_table.lookup(char_tensor)

#         token_ids_list.append(token_ids)

#     # Pad all sequences to the same length
#     token_ids_padded = tf.ragged.stack(token_ids_list).to_tensor(default_value=pad_value) # type: ignore
#     # Create attention mask: 1 for real tokens, 0 for padding
#     attention_mask = tf.cast(token_ids_padded != pad_value, tf.int32)

#     return token_ids_padded, attention_mask


In [7]:
# import tensorflow as tf
# from typing import Tuple


# def tokenize_and_build_token_id(
#     token_to_id_dict: dict,
#     text_batch: list[str],
#     max_seq_len: int,
#     pad_value: int = 0,
#     unk_value: int = None
# ) -> Tuple[tf.Tensor, tf.Tensor]:
#     """
#     TensorFlow-compatible tokenization converting batch of strings to char token IDs,
#     padded/truncated to max_seq_len, along with attention mask.

#     Args:
#         token_to_id_dict: dict mapping character str -> int ID
#         text_batch: list of strings to tokenize
#         max_seq_len: max length to pad/truncate sequences
#         pad_value: int ID for padding tokens
#         unk_value: int ID for unknown tokens; if None, uses pad_value

#     Returns:
#         token_ids: (batch_size, max_seq_len) tf.int32 tensor of token IDs
#         attention_mask: (batch_size, max_seq_len) tf.int32 tensor (1 for tokens, 0 for padding)
#     """

#     if unk_value is None:
#         unk_value = pad_value

#     # Create lookup table from token_to_id_dict
#     keys = tf.constant(list(token_to_id_dict.keys()))
#     values = tf.constant(list(token_to_id_dict.values()), dtype=tf.int32)
#     table = tf.lookup.StaticHashTable(
#         tf.lookup.KeyValueTensorInitializer(keys, values),
#         default_value=unk_value
#     )

#     # Convert text batch to a RaggedTensor of chars
#     rt_chars = tf.strings.unicode_split(text_batch, 'UTF-8')  # shape: [batch_size, (seq_len)]

#     # Lookup token IDs for each char
#     token_ids = table.lookup(rt_chars)

#     # Pad or truncate sequences to max_seq_len
#     token_ids = token_ids.to_tensor(default_value=pad_value, shape=[None, max_seq_len])
#     token_ids = token_ids[:, :max_seq_len]  # truncate if longer

#     # Construct attention mask: 1 where not pad_value, else 0
#     attention_mask = tf.cast(token_ids != pad_value, tf.int32)

#     return token_ids, attention_mask


In [8]:
import os
import re
from typing import List, Dict, Tuple
from collections import Counter
import tensorflow as tf
import numpy as np

@keras.saving.register_keras_serializable()
def tokenize_and_build_vocabulary_tf(
    file_path_list: List[str],
    vocab_size: int = 10000,
    existing_vocab: Dict[str,int]|None = None
) -> Dict[str,int]:
    """
    Build a word-level vocabulary from text files with size control.
    
    Args:
        file_path_list: List of file paths to process
        vocab_size: Maximum vocabulary size (includes special tokens)
        existing_vocab: Optional existing vocabulary to extend
        
    Returns:
        token_to_id mapping each word (or <UNK>, <PAD>) to an integer ID.
    """
    if isinstance(file_path_list, (str, bytes)):
        file_path_list = [file_path_list]  # type: ignore
    
    if existing_vocab is None:
        existing_vocab = {'<PAD>': 0, '<UNK>': 1}
    
    # Count word frequencies across all files
    word_counts = Counter()
    
    for fn in file_path_list:
        if os.path.isdir(fn):
            raise IsADirectoryError(fn)
        if not os.path.isfile(fn):
            raise FileNotFoundError(fn)
        
        with open(fn, encoding='utf-8') as f:
            text = f.read().lower()
        
        # Simple word tokenizer; keeps only alphanumeric words
        words = re.findall(r"\b\w+\b", text)
        word_counts.update(words)
    
    # Remove special tokens from word counts if they exist
    for special_token in ['<PAD>', '<UNK>']:
        if special_token in word_counts:
            del word_counts[special_token]
    
    # Get the most common words (excluding space for special tokens)
    num_special_tokens = len(existing_vocab)
    max_regular_words = vocab_size - num_special_tokens
    
    most_common_words = [word for word, _ in word_counts.most_common(max_regular_words)]
    
    # Build the final vocabulary
    token_to_id = existing_vocab.copy()
    
    for idx, word in enumerate(most_common_words, start=num_special_tokens):
        token_to_id[word] = idx
    
    print(f"Built vocabulary with {len(token_to_id)} tokens "
          f"({num_special_tokens} special + {len(most_common_words)} regular)")
    
    return token_to_id


@keras.saving.register_keras_serializable()
def tokenize_and_build_token_id(
    token_to_id_dict: Dict[str,int],
    text_batch: List[str],
    max_seq_len: int,
    pad_value: int = 0
) -> Tuple[tf.Tensor, tf.Tensor]:
    '''
    Tokenize batch of strings into word-IDs, pad/truncate, and build attention masks.
    
    Args:
        token_to_id_dict: Dictionary mapping tokens to IDs
        text_batch: List of text strings to tokenize
        max_seq_len: Maximum sequence length (pad/truncate to this length)
        pad_value: Value to use for padding (should match <PAD> token ID)
        
    Returns:
        Tuple of (token_ids, attention_mask) as TensorFlow tensors
    '''
    batch_ids = []
    
    for text in text_batch:
        words = re.findall(r"\b\w+\b", text.lower())
        ids = [token_to_id_dict.get(w, token_to_id_dict['<UNK>']) for w in words]
        
        # Truncate if too long
        if len(ids) > max_seq_len:
            ids = ids[:max_seq_len]
        # Pad if too short
        else:
            ids += [pad_value] * (max_seq_len - len(ids))
        
        batch_ids.append(ids)

    token_ids = np.array(batch_ids, dtype=np.int32)
    attention_mask = (token_ids != pad_value).astype(np.int32)

    return tf.constant(token_ids), tf.constant(attention_mask)


In [9]:
token_to_id_dict = tokenize_and_build_vocabulary_tf(['/home/akshat/GPT_from_scratch/text_data/jane_austen_clean.txt'],vocab_size = 5000)
token_to_id_dict

Built vocabulary with 5000 tokens (2 special + 4998 regular)


{'<PAD>': 0,
 '<UNK>': 1,
 'the': 2,
 'to': 3,
 'and': 4,
 'of': 5,
 'a': 6,
 'her': 7,
 'i': 8,
 'in': 9,
 'was': 10,
 'it': 11,
 'she': 12,
 'not': 13,
 'that': 14,
 'be': 15,
 'you': 16,
 'he': 17,
 'had': 18,
 'as': 19,
 'for': 20,
 'with': 21,
 'his': 22,
 'but': 23,
 'have': 24,
 'is': 25,
 'at': 26,
 's': 27,
 'so': 28,
 'my': 29,
 'all': 30,
 'on': 31,
 'very': 32,
 'by': 33,
 'him': 34,
 'could': 35,
 'which': 36,
 'been': 37,
 'would': 38,
 'no': 39,
 'were': 40,
 'they': 41,
 'mr': 42,
 'from': 43,
 'this': 44,
 'me': 45,
 'what': 46,
 'mrs': 47,
 'will': 48,
 'do': 49,
 'or': 50,
 'an': 51,
 'them': 52,
 'more': 53,
 'there': 54,
 'such': 55,
 'their': 56,
 'must': 57,
 'your': 58,
 'any': 59,
 'said': 60,
 'if': 61,
 'one': 62,
 'much': 63,
 'when': 64,
 'than': 65,
 'are': 66,
 'am': 67,
 'miss': 68,
 'we': 69,
 'only': 70,
 'should': 71,
 'who': 72,
 'well': 73,
 'did': 74,
 'every': 75,
 'being': 76,
 'now': 77,
 'how': 78,
 'think': 79,
 'own': 80,
 'never': 81,
 'know

In [10]:
name = tf.constant('Akshat Khatri')
tf.strings.bytes_split(name)

I0000 00:00:1756542260.516338    1336 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


<tf.Tensor: shape=(13,), dtype=string, numpy=
array([b'A', b'k', b's', b'h', b'a', b't', b' ', b'K', b'h', b'a', b't',
       b'r', b'i'], dtype=object)>

In [11]:
name = tf.constant(['Hello','Worlds '])
for name in name.numpy():
    print(name)

b'Hello'
b'Worlds '


In [12]:
a = tf.constant([1.,2.,3.])
type(a)

tensorflow.python.framework.ops.EagerTensor

In [13]:
name = ['Akshat Khatri','hello','me','elizabeth','children','mine','coherent','i am fine','Children','trained','model']
tokenize_and_build_token_id(token_to_id_dict,name,512)

(<tf.Tensor: shape=(11, 512), dtype=int32, numpy=
 array([[   1,    1,    0, ...,    0,    0,    0],
        [   1,    0,    0, ...,    0,    0,    0],
        [  45,    0,    0, ...,    0,    0,    0],
        ...,
        [ 389,    0,    0, ...,    0,    0,    0],
        [   1,    0,    0, ...,    0,    0,    0],
        [3304,    0,    0, ...,    0,    0,    0]],
       shape=(11, 512), dtype=int32)>,
 <tf.Tensor: shape=(11, 512), dtype=int32, numpy=
 array([[1, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]], shape=(11, 512), dtype=int32)>)

In [14]:
import tensorflow as tf
print(tf.__version__)

2.20.0


In [15]:
# from typing import Callable

# class InitializePositionalEmbeddings(keras.layers.Layer): # Receives input of sequence of text
#     def __init__(self,d_model: int = 128,sinusoidal_lookup_table = [],token_to_id_dict : tf.lookup.StaticHashTable = {} ,max_seq_len : int = 512,**kwargs): # type: ignore
#         super().__init__(**kwargs)
#         self.d_model = d_model # d_model
#         self.max_seq_len = max_seq_len
        
#         assert len(sinusoidal_lookup_table) > 0
#         assert token_to_id_dict.size().numpy() > 0
#         self.VOCAB_SIZE = token_to_id_dict.size().numpy()

#         self.pos_table = sinusoidal_lookup_table
#         self._embedding_dim = [self.VOCAB_SIZE,d_model]
#         self.token_to_id_dict = token_to_id_dict
    
#     def build(self, input_shape): # this is batch input shape
#         print(input_shape)
#         self.embedding_matrix = self.add_weight(
#             name="embedding_matrix",
#             shape=(self.VOCAB_SIZE, self.d_model),
#             initializer="random_normal",
#             trainable=True   # important
#         )
#         self.input_seq_list = input_shape[-1]

#     def call(self,inputs):
#         # print(inputs)
#         tokens_in_id,non_padded_tokens_mask = tokenize_and_build_token_id(self.token_to_id_dict,inputs)
#         # print(tokens_in_id,non_padded_tokens_mask,sep = '\n')
#         token_embeddings = tf.nn.embedding_lookup(self.embedding_matrix, tokens_in_id)
#         # Positional embeddings
#         seq_len = tf.shape(tokens_in_id)[1] # type: ignore
#         pos_embeddings = self.pos_table[:seq_len, :]
#         pos_embeddings = tf.expand_dims(pos_embeddings, 0)  # broadcast along batch
#         # Add token + position embeddings
#         embeddings = token_embeddings + pos_embeddings
#         return embeddings,non_padded_tokens_mask
    
#     def get_config(self):
#         base_config = super().get_config()
#         return {**base_config,'EMBEDDING_SIZE' : self.EMBEDDING_SIZE,'VOCAB_SIZE' : self.VOCAB_SIZE}


In [16]:
@keras.saving.register_keras_serializable()
class InitializePositionalEmbeddings(keras.layers.Layer):
    def __init__(
        self,
        d_model: int,
        vocab_size : int,
        CONTEXT_LEN: int = 128,
        pad_value: int = 0,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.d_model = int(d_model)
        self.pad_value = int(pad_value)
        self.vocab_size = vocab_size
        self._pos_table = prepare_sinusoidal_lookup_table(d_model, CONTEXT_LEN)

    def build(self, input_shape):
        self.embedding_matrix = self.add_weight(
            name="embedding_matrix",
            shape=(self.vocab_size, self.d_model),
            initializer="random_normal",
            trainable=True,
            dtype=tf.float32
        )
        super().build(input_shape)

    def call(self, text_batch):

        token_ids= text_batch # Unpacking Data Pre-processing inputs Embeddings
        
        # Embeddings lookup: (B, T, D)
        token_emb = tf.nn.embedding_lookup(self.embedding_matrix, token_ids)
        # Positional embeddings: slice and broadcast
        seq_len = tf.shape(token_ids)[1] # type: ignore
        pos_emb = self._pos_table[:seq_len, :]    # type: ignore # (T, D)
        pos_emb = tf.expand_dims(pos_emb, 0)     # (1, T, D)
        embeddings = token_emb + pos_emb         # (B, T, D)
        return embeddings

    # def compute_output_shape(self, input_shape):
    #     # input_shape: (batch_size,)
    #     batch = input_shape
    #     # Sequence length is dynamic: None
    #     return (batch, None, self.d_model), (batch, None)

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "d_model": self.d_model,
            'vocab_size': self.vocab_size,
            'max_seq_len': self.max_seq_len,
            "pad_value": self.pad_value,
        })
        return cfg
    
    def compute_output_shape(self, input_shape):
        # input_shape: (batch_size, seq_len)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        return (batch_size, seq_len, self.d_model)


In [17]:
VOCAB_SIZE = len(token_to_id_dict)
D_MODEL = 1024
MAX_SEQ_LEN = 512
VOCAB_SIZE

5000

In [18]:
sinusoidal_lookup_table = prepare_sinusoidal_lookup_table(D_MODEL)
batch_text = ['yo','Akshat Khatri', 'Hello World','Me']
batch_text = tokenize_and_build_token_id(token_to_id_dict,batch_text,MAX_SEQ_LEN) # type: ignore
token_ids,attention_mask = batch_text

layer = InitializePositionalEmbeddings(D_MODEL,VOCAB_SIZE)

# @tf.function
# def call_some(batch_text):
#     embeddings = layer(batch_text)
#     return embeddings

# call_some(token_ids)

In [19]:
@keras.saving.register_keras_serializable()
class SelfAttentionLayer(keras.layers.Layer):
    def __init__(self, attention_heads=4, **kwargs):
        super().__init__(**kwargs)
        self.attention_heads = attention_heads
        
    def build(self, input_shape):
        self.d_model = input_shape[0][-1]
        
        self.Query_projection = self.add_weight(
            name='Query_Vector_for_projection',
            initializer='random_normal',
            shape=(self.d_model, self.d_model),
            trainable=True 
        )
        self.Key_projection = self.add_weight(
            name='Key_Vector_for_projection',
            initializer='random_normal',
            shape=(self.d_model, self.d_model),
            trainable=True 
        )
        self.Value_projection = self.add_weight(
            name='Value_Vector_for_projection',
            initializer='random_normal',
            shape=(self.d_model, self.d_model),
            trainable=True 
        )
        self.output_projection = self.add_weight(
            name="Output_projection",
            initializer="random_normal",
            shape=(self.d_model, self.d_model),
            trainable=True,
        )

        self.d_head = self.d_model // self.attention_heads
        assert self.d_model % self.attention_heads == 0, "d_model must be divisible by attention_heads"

    def call(self, inputs):
        embeddings = inputs[0]
        token_masks = inputs[1]

        batch_size = tf.shape(embeddings)[0]
        seq_len = tf.shape(embeddings)[1]

        # 1. Project to Q, K, V
        Q = embeddings @ self.Query_projection
        K = embeddings @ self.Key_projection
        V = embeddings @ self.Value_projection

        # 2. Reshape for multi-head attention
        Q = tf.reshape(Q, (batch_size, seq_len, self.attention_heads, self.d_head))
        K = tf.reshape(K, (batch_size, seq_len, self.attention_heads, self.d_head))
        V = tf.reshape(V, (batch_size, seq_len, self.attention_heads, self.d_head))

        Q = tf.transpose(Q, (0, 2, 1, 3))  # (batch, heads, seq_len, d_head)
        K = tf.transpose(K, (0, 2, 1, 3))
        V = tf.transpose(V, (0, 2, 1, 3))

        # 3. Compute attention scores
        scores = tf.matmul(Q, K, transpose_b=True)  # (batch, heads, seq_len, seq_len)
        scores = scores / tf.sqrt(tf.cast(self.d_head, tf.float32))
        
        # 4. FIXED MASKING - This was your main bug
        # 4a. Causal mask (L,L) lower triangular
        causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        
        # 4b. Token mask - FIXED: proper broadcasting to all heads
        token_mask = tf.cast(token_masks, tf.float32)  # (B, L)
        
        # Create proper attention mask shape (B, H, L, L)
        # Each head gets the same mask pattern
        attention_mask = causal_mask[tf.newaxis, tf.newaxis, :, :]  # (1, 1, L, L)
        attention_mask = attention_mask * token_mask[:, tf.newaxis, tf.newaxis, :]  # (B, 1, 1, L)
        attention_mask = attention_mask * token_mask[:, tf.newaxis, :, tf.newaxis]  # (B, 1, L, 1)
        
        # Broadcast to all heads
        attention_mask = tf.broadcast_to(attention_mask, (batch_size, self.attention_heads, seq_len, seq_len))
        
        # 5. Apply mask with stronger negative value
        scores = tf.where(
            attention_mask > 0, 
            scores, 
            tf.constant(-1e30, dtype=scores.dtype)  # FIXED: Much more negative
        )

        # 6. Softmax and apply to values
        attention_weights = tf.nn.softmax(scores, axis=-1)
        
        # Add attention dropout (missing in your original)
        attention_weights = tf.nn.dropout(attention_weights, rate=0.1)
        
        context = attention_weights @ V   # (batch, heads, seq_len, d_head)
        
        # 7. Concatenate heads
        concat_context = tf.transpose(context, (0, 2, 1, 3))  # (batch, seq_len, heads, d_head)
        concat_context = tf.reshape(concat_context, (batch_size, seq_len, self.d_model))
        
        # 8. Final projection
        final_context = concat_context @ self.output_projection 
        return final_context
    
    def get_config(self):
        config = super().get_config()
        config.update({"attention_heads": self.attention_heads})
        return config
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]


In [20]:
import numpy as np
import tensorflow as tf

a = np.random.rand(3, 13, 64)  # batch, seq_len, d_model
q = np.ones((64, 64))          # d_model × d_model

a = tf.constant(a, dtype=tf.float32)
q = tf.constant(q, dtype=tf.float32)

s = a @ q   # type: ignore # [3, 13, 64]
d_head = 16
num_heads = 64 // d_head # 4

# split into heads
s = tf.reshape(s, (3, num_heads, 13, d_head))  # [3, 4, 13, 16]
f = tf.constant(np.ones_like(s))

tf.transpose(f)


<tf.Tensor: shape=(16, 13, 4, 3), dtype=float32, numpy=
array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]],


       [[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],

In [21]:
arr = tf.constant([[1,2,9,4,5],[4,5,6,7,8]])
keras.activations.softmax(tf.cast(arr,dtype = tf.float32),axis = 0)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.04742587, 0.04742587, 0.95257413, 0.04742587, 0.04742587],
       [0.95257413, 0.95257413, 0.04742587, 0.95257413, 0.95257413]],
      dtype=float32)>

In [22]:
arr = tf.constant([[1,2,3,4,5],[4,5,6,7,8]])
arr = tf.cast(arr,dtype = tf.float32)

tf.nn.softmax(arr,axis = 0)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[0.04742587, 0.04742587, 0.04742587, 0.04742587, 0.04742587],
       [0.95257413, 0.95257413, 0.95257413, 0.95257413, 0.95257413]],
      dtype=float32)>

In [23]:
-1e9

-1000000000.0

In [24]:
@keras.saving.register_keras_serializable()
class LayerNormalization(keras.layers.Layer):
    def __init__(self,eps=1e-5,**kwargs):
        super().__init__(**kwargs)
        self.eps = eps
    
    def build(self,input_shape): # Near Attention (batch, seq_len, d_model)
        self.alpha = self.add_weight(
            name = 'alpha',
            shape = input_shape[-1:],
            initializer = 'ones',
            dtype = tf.float32,
            trainable = True
        )
        self.beta = self.add_weight(
            name = 'beta',
            shape = input_shape[-1:],
            initializer = 'zeros',
            dtype = tf.float32,
            trainable = True
        )
        super().build(input_shape)
        
    def call(self, inputs):
        mean, var = tf.nn.moments(inputs, axes=[-1], keepdims=True)
        normed = (inputs - mean) / tf.sqrt(var + self.eps) # type: ignore
        return self.alpha * normed + self.beta

    def get_config(self):
        base = super().get_config()
        return {**base, "eps": self.eps}
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [25]:
class SimpleDense(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="he_normal",
            trainable=True,
            name="kernel",
        )
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer="zeros",
            trainable=True,
            name="bias",
        )
        super().build(input_shape)

    def call(self, inputs):
        # inputs shape: (batch, seq_len, input_dim)
        output = tf.matmul(inputs, self.kernel) + self.bias  # shape: (batch, seq_len, units))  # shape: (batch, seq_len, units)
        return output + self.bias

    def compute_output_shape(self, input_shape):
        # input_shape: (batch, seq_len, input_dim)
        return (input_shape, input_shape[1], self.units)

    def get_config(self):
        base = super().get_config()
        return {**base, "units": self.units}

In [26]:
a = (2,3,4,5,6)
a[-1:]

(6,)

In [27]:
@keras.saving.register_keras_serializable()
class DecoderBlock(keras.Model):
    '''A single Decoder Block'''
    def __init__(self, d_model, n_heads, dropout_rate=0.1, epsilon=1e-5, name=None):
        super().__init__(name=name)
        self.d_model = d_model
        self.n_heads = n_heads
        self.dropout_rate = dropout_rate
        self.epsilon = epsilon
        # norms
        self.ln1 = LayerNormalization(epsilon)   # pre-attn
        self.ln2 = LayerNormalization(epsilon)   # pre-ffn
        # attention (assumes your SelfAttentionLayer accepts (x, attention_mask))
        self.attn = SelfAttentionLayer(n_heads)
        self.dropout1 = keras.layers.Dropout(dropout_rate)
        # FFN
        self.ffn1 = keras.layers.Dense(4 * d_model, activation="gelu")
        self.ffn2 = keras.layers.Dense(d_model)
        self.dropout2 = keras.layers.Dropout(dropout_rate)

    def call(self, x, attention_mask, training=False):
        # Self-attention sublayer
        y = self.ln1(x)
        y = self.attn((y, attention_mask))          # shape: (B, T, d_model)
        y = self.dropout1(y, training=training)
        x = x + y                                    # residual

        # FFN sublayer
        y = self.ln2(x)
        y = self.ffn1(y)
        y = self.ffn2(y)
        y = self.dropout2(y, training=training)
        x = x + y                                    # residual
        return x
    
    def compute_output_shape(self, input_shape):
        # input_shape is typically (batch_size, seq_len, d_model)
        return input_shape
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "d_model": self.d_model,
            "n_heads": self.n_heads,
            "dropout_rate": self.dropout_rate,
            "epsilon": self.epsilon,
        })
        return config

@keras.saving.register_keras_serializable()
class GPT(keras.Model):
    '''
    GPT model with N distinct blocks
      -----------------------------------'''
    def __init__(self,
                 d_model: int = 128,
                 vocab_size: int = 94,
                 context_length: int = 512,
                 attention_heads: int = 8,
                 epsilon: float = 1e-5,
                 decoder_blocks: int = 3,
                 dropout_rate: float = 0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self._d_model = d_model
        self._vocab_size = vocab_size
        self._context_length = context_length
        self._attention_heads = attention_heads
        self._epsilon = epsilon
        self._decoder_blocks = decoder_blocks
        self._dropout_rate = dropout_rate

        # embeddings (yours)
        self.emb = InitializePositionalEmbeddings(
            d_model, vocab_size,context_length,name="init_embeddings"
        )

        # stack of distinct decoder blocks
        self.blocks = [
            DecoderBlock(d_model, attention_heads, dropout_rate, epsilon, name=f"decoder_block_{i}")
            for i in range(decoder_blocks)
        ]

        # final norm (GPT-2 style) and LM head
        self.final_ln = LayerNormalization(epsilon)
        self.lm_head = keras.layers.Dense(vocab_size, name="Model_head")

    def call(self, inputs, training=False):
        """
        inputs: (token_ids, attention_mask)
          - token_ids: int32 (B, T)
          - attention_mask: int32/float32 mask broadcasting to attention logits.
            Common shapes: (B, 1, 1, T) or (B, T) if your SelfAttentionLayer handles expansion.
        """
        token_ids, attention_mask = inputs
        x = self.emb(token_ids)                         # (B, T, d_model)

        for block in self.blocks:
            x = block(x, attention_mask, training=training)

        x = self.final_ln(x)
        logits = self.lm_head(x)                        # (B, T, vocab_size)
        return logits                                   # keep softmax outside

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            "d_model": self._d_model,
            "vocab_size": self._vocab_size,
            "context_length": self._context_length,
            "attention_heads": self._attention_heads,
            "epsilon": self._epsilon,
            "decoder_blocks": self._decoder_blocks,
            "dropout_rate": self._dropout_rate,
        })
        return cfg


In [28]:
CONTEXT_LEN = 64
VOCAB_SIZE = len(token_to_id_dict) # 94 currently char level

batch_text = ['yo','Akshat Khatri', 'Hello World','Me']
token_ids,attention_mask = tokenize_and_build_token_id(token_to_id_dict,batch_text,CONTEXT_LEN) # type: ignore # Unpacking Values
sinusoidal_lookup_table = prepare_sinusoidal_lookup_table(D_MODEL,CONTEXT_LEN)

In [29]:
[token_ids,attention_mask]

[<tf.Tensor: shape=(4, 64), dtype=int32, numpy=
 array([[  1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  1,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  1, 216,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0

In [30]:
@keras.saving.register_keras_serializable()
class CosineDecayWithWarmup(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, 
                 warmup_steps: int,
                 total_steps: int,
                 peak_learning_rate: float = 1e-4,
                 min_learning_rate: float = 1e-6,
                 name: str = "cosine_decay_with_warmup"):
        super().__init__()
        self.warmup_steps = warmup_steps
        self.total_steps = total_steps
        self.peak_learning_rate = peak_learning_rate
        self.min_learning_rate = min_learning_rate
        self.name = name
        
    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        total_steps = tf.cast(self.total_steps, tf.float32)
        
        # Warmup phase: linear increase from 0 to peak_learning_rate
        warmup_lr = self.peak_learning_rate * step / warmup_steps
        
        # Cosine decay phase
        decay_steps = total_steps - warmup_steps
        cosine_decay_lr = self.min_learning_rate + 0.5 * (
            self.peak_learning_rate - self.min_learning_rate
        ) * (1 + tf.cos(np.pi * (step - warmup_steps) / decay_steps))
        
        return tf.where(step < warmup_steps, warmup_lr, cosine_decay_lr)
    
    def get_config(self):
        return {
            "warmup_steps": self.warmup_steps,
            "total_steps": self.total_steps,
            "peak_learning_rate": self.peak_learning_rate,
            "min_learning_rate": self.min_learning_rate,
            "name": self.name,
        }

# Example usage for your model
# Estimate your training parameters
EPOCHS = 20
STEPS_PER_EPOCH = 1000  # Adjust based on your dataset size and batch size
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH
WARMUP_STEPS = int(0.1 * TOTAL_STEPS)  # 10% warmup

# Create the learning rate schedule
lr_schedule = CosineDecayWithWarmup(
    warmup_steps=WARMUP_STEPS,
    total_steps=TOTAL_STEPS,
    peak_learning_rate=1e-4,  # Your desired peak learning rate
    min_learning_rate=1e-6    # Minimum learning rate at the end
)


## Modify above according to requirements

In [31]:
# build once to get .summary()
DECODER_BLOCKS = 8
ATTENTION_HEADS = 4

GPT_model = GPT(D_MODEL,VOCAB_SIZE,CONTEXT_LEN,ATTENTION_HEADS,0.00001,DECODER_BLOCKS,0.1)
_ = GPT_model((token_ids, attention_mask))
GPT_model.summary(expand_nested=True)

# training (stable): use logits
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = keras.optimizers.AdamW(learning_rate=lr_schedule, weight_decay=1e-4, clipnorm=1.0)
GPT_model.compile(optimizer=opt, loss=loss) # type: ignore

# inference probs (when you actually need them)
logits = GPT_model((token_ids, attention_mask), training=False)
probs = keras.ops.softmax(logits, axis=-1)


Model: "gpt"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ init_embeddings                 │ (4, 64, 1024)          │     5,120,000 │
│ (InitializePositionalEmbedding… │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_0 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ layer_normalization        │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ layer_normalization_1      │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ self_attention_layer       │ (4, 64, 1024)          │     4,194,304 │
│ (SelfAttentionLayer)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dropout (Dropout)          │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense (Dense)              │ (4, 64, 4096)          │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_1 (Dense)            │ (4, 64, 1024)          │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dropout_1 (Dropout)        │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_1 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ layer_normalization_2      │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ layer_normalization_3      │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ self_attention_layer_1     │ (4, 64, 1024)          │     4,194,304 │
│ (SelfAttentionLayer)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dropout_2 (Dropout)        │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_2 (Dense)            │ (4, 64, 4096)          │     4,198,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dense_3 (Dense)            │ (4, 64, 1024)          │     4,195,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dropout_3 (Dropout)        │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_2 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ layer_normalization_4      │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ layer_normalization_5      │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 110,984,072 (423.37 MB)

 Trainable params: 110,984,072 (423.37 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
probs

<tf.Tensor: shape=(4, 64, 5000), dtype=float32, numpy=
array([[[9.17721263e-05, 1.58267620e-04, 7.89049882e-05, ...,
         4.62345524e-05, 1.11462432e-04, 2.63327558e-04],
        [2.16269196e-04, 3.55634198e-04, 1.23658232e-04, ...,
         1.27934618e-04, 1.64374433e-04, 1.01247846e-04],
        [2.25262251e-04, 3.35881399e-04, 1.28588930e-04, ...,
         1.23376216e-04, 1.56066497e-04, 1.05651168e-04],
        ...,
        [2.38189517e-04, 3.20498715e-04, 1.23040430e-04, ...,
         1.24532031e-04, 1.51542234e-04, 1.22151483e-04],
        [2.32070524e-04, 3.26718087e-04, 1.24747923e-04, ...,
         1.22275713e-04, 1.55416099e-04, 1.22848462e-04],
        [2.32014121e-04, 3.25196423e-04, 1.19121287e-04, ...,
         1.19630742e-04, 1.57797651e-04, 1.21494486e-04]],

       [[1.21259392e-04, 1.85770652e-04, 9.58174496e-05, ...,
         5.75506056e-05, 1.00423167e-04, 3.54145566e-04],
        [1.09879955e-04, 2.03619085e-04, 8.85180925e-05, ...,
         4.91684514e-05, 1.0

In [33]:
outputs = GPT_model((token_ids, attention_mask))
probs = tf.nn.softmax(outputs, axis=-1)
probs

<tf.Tensor: shape=(4, 64, 5000), dtype=float32, numpy=
array([[[1.12514266e-04, 2.23259922e-04, 6.15297249e-05, ...,
         6.33913223e-05, 1.57999486e-04, 1.34142581e-04],
        [2.24163901e-04, 3.53680662e-04, 1.30546076e-04, ...,
         1.20510158e-04, 1.67891340e-04, 9.84475919e-05],
        [2.23144059e-04, 3.37189354e-04, 1.26456565e-04, ...,
         1.22453566e-04, 1.62833108e-04, 1.00729550e-04],
        ...,
        [2.32919265e-04, 3.39150371e-04, 1.21412821e-04, ...,
         1.25228340e-04, 1.51621061e-04, 1.20937577e-04],
        [2.31672486e-04, 3.44316504e-04, 1.17227486e-04, ...,
         1.23874546e-04, 1.63023476e-04, 1.11811649e-04],
        [2.25805503e-04, 3.47522349e-04, 1.23126199e-04, ...,
         1.23148275e-04, 1.69446255e-04, 1.13670518e-04]],

       [[1.33764363e-04, 2.00560782e-04, 1.13622555e-04, ...,
         6.27676636e-05, 8.23706287e-05, 3.12353688e-04],
        [9.59440003e-05, 2.28337434e-04, 1.06569569e-04, ...,
         4.90231469e-05, 1.5

In [34]:
GPT_model.summary()

Model: "gpt"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ init_embeddings                 │ (4, 64, 1024)          │     5,120,000 │
│ (InitializePositionalEmbedding… │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_0 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_1 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_2 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_3 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_4 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_5 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_6 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_block_7 (DecoderBlock)  │ (4, 64, 1024)          │    12,592,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_16          │ (4, 64, 1024)          │         2,048 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Model_head (Dense)              │ (4, 64, 5000)          │     5,125,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,984,072 (423.37 MB)

 Trainable params: 110,984,072 (423.37 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
from keras.utils import plot_model

try:
    plot_model(
        GPT_model,
        to_file="gpt_model.png",
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True,
        dpi=160
    )
    print("Saved diagram to gpt_model.png")
except Exception as e:
    print("plot_model failed:", e)

Saved diagram to gpt_model.png


In [36]:
GPT_model.layers

[<InitializePositionalEmbeddings name=init_embeddings, built=True>,
 <DecoderBlock name=decoder_block_0, built=True>,
 <DecoderBlock name=decoder_block_1, built=True>,
 <DecoderBlock name=decoder_block_2, built=True>,
 <DecoderBlock name=decoder_block_3, built=True>,
 <DecoderBlock name=decoder_block_4, built=True>,
 <DecoderBlock name=decoder_block_5, built=True>,
 <DecoderBlock name=decoder_block_6, built=True>,
 <DecoderBlock name=decoder_block_7, built=True>,
 <LayerNormalization name=layer_normalization_16, built=True>,
 <Dense name=Model_head, built=True>]

In [37]:
GPT_model.count_params()

110984072

In [38]:
import numpy as np

def format_model_summary(model):
    print("="*80)
    print(f"{'GPT MODEL SUMMARY':^80}")
    print("="*80)
    
    total_params = model.count_params()
    total_layers = len(model.layers)
    total_weights = sum(len(layer.trainable_weights) for layer in model.layers)
    try:
        output_shapes = [tuple(out.shape) for out in model.outputs]
    except Exception:
        output_shapes = ["(unavailable)"]
    
    print(f"{'Total parameters:':<22} {total_params:,}")
    print(f"{'Total layers:':<22} {total_layers}")
    print(f"{'Trainable weights:':<22} {total_weights}")
    print(f"{'Final output shape(s):':<22} {output_shapes if output_shapes else '(N/A)'}")
    print("-"*80)
    
    header = f"{'Idx':>3} | {'Layer Type':<24} | {'Layer Name':<23} | {'Weight Name':<28} | {'Shape':<15} | {'Params':>8}"
    print(header)
    print("-" * len(header))
    
    for i, layer in enumerate(model.layers):
        layer_type = layer.__class__.__name__
        layer_name = layer.name
        weights = layer.trainable_weights
        layer_weight_count = len(weights)

        if layer_weight_count == 0:
            print(f"{i:03} | {layer_type:<24} | {layer_name:<23} | {'-':<28} | {'-':<15} | {'0':>8}")
        else:
            for j, w in enumerate(weights):
                n = int(np.prod(w.shape)) if hasattr(w, "shape") else "?"
                shape_str = str(tuple(w.shape))
                weight_name = w.name
                if j == 0:
                    print(f"{i:03} | {layer_type:<24} | {layer_name:<23} | {weight_name:<28} | {shape_str:<15} | {n:>8,}")
                else:
                    print(f"    | {'':<24} | {'':<23} | {weight_name:<28} | {shape_str:<15} | {n:>8,}")
        if layer_weight_count > 1:
            print(f"    | {'':<24} | {'':<23} | {'':<28} | {'':<15} | {'':>8}")

    print("="*80)
    print("Note: Only trainable weights are listed above. Output shapes may be unavailable\n"
          "for subclassed models or models not built symbolically.")
    print("="*80 + "\n")

# Usage:
format_model_summary(GPT_model)


                               GPT MODEL SUMMARY                                
Total parameters:      110,984,072
Total layers:          11
Trainable weights:     101
Final output shape(s): ['(unavailable)']
--------------------------------------------------------------------------------
Idx | Layer Type               | Layer Name              | Weight Name                  | Shape           |   Params
--------------------------------------------------------------------------------------------------------------------
000 | InitializePositionalEmbeddings | init_embeddings         | embedding_matrix             | (5000, 1024)    | 5,120,000
001 | DecoderBlock             | decoder_block_0         | alpha                        | (1024,)         |    1,024
    |                          |                         | beta                         | (1024,)         |    1,024
    |                          |                         | alpha                        | (1024,)         |    1,024


In [39]:
# def prepare_book_training_data(book_text: str, 
#                              token_to_id_dict: Dict[str, int], 
#                              context_length: int = 512,
#                              pad_value: int = 0):
#     """
#     Prepare training data from a Gutenberg book
#     """
#     # 1. Tokenize the entire book
#     token_ids = [token_to_id_dict.get(c, pad_value) for c in book_text]
    
#     # 2. Create sliding windows
#     inputs = []
#     targets = []
    
#     # Slide window across the entire book
#     for i in range(0, len(token_ids) - context_length, context_length):
#         # Extract window of context_length + 1 tokens
#         window = token_ids[i:i + context_length + 1]
        
#         if len(window) < context_length + 1:
#             break  # Skip incomplete windows at the end
        
#         # Create input-target pair
#         input_seq = window[:-1]   # [t1, t2, ..., t512]
#         target_seq = window[1:]   # [t2, t3, ..., t513]
        
#         inputs.append(input_seq)
#         targets.append(target_seq)
    
#     # 3. Convert to numpy arrays
#     inputs = np.array(inputs, dtype=np.int32)
#     targets = np.array(targets, dtype=np.int32)
    
#     # 4. Create padding masks (all 1s since we're using full context)
#     masks = np.ones_like(inputs, dtype=np.int32)
    
#     return inputs, targets, masks

# # Usage:
# with open(r'/home/akshat/GPT_from_scratch/text_data/pg76702.txt', 'r', encoding='utf-8') as f:
#     book_text = f.read()

# inputs, targets, masks = prepare_book_training_data(
#     book_text, 
#     token_to_id_dict, 
#     context_length=512
# )

# print(f"Created {len(inputs)} training examples")
# print(f"Input shape: {inputs.shape}")
# print(f"Target shape: {targets.shape}")

In [40]:
# with open(r'/home/akshat/GPT_from_scratch/text_data/pg76702.txt', 'r') as f:
#     book_text = f.read()

# print(f"Total characters: {len(book_text)}")
# print(f"Expected examples (rough): {len(book_text) // 512}")

''' Stop '''

In [41]:
# # Convert to TensorFlow tensors
# input_ids = tf.constant(inputs, dtype=tf.int32)
# target_ids = tf.constant(targets, dtype=tf.int32)
# attention_masks = tf.constant(masks, dtype=tf.int32)

# model = GPT(D_MODEL,VOCAB_SIZE,CONTEXT_LEN,8,0.00001,4,0.1,sinusoidal_lookup_table)

# # Compile your model
# model.compile(
#     optimizer=keras.optimizers.AdamW(learning_rate=1e-4), # type: ignore
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )

# # Train with .fit()
# history = model.fit(
#     x=[input_ids, ,  # Your model expects (token_ids, attention_mask)
#     y=target_ids,
#     batch_size=16,  # Start small since 677 examples isn't huge
#     epochs=50,
#     validation_split=0.1,  # Use 10% for validation
#     callbacks=[
#         keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True),
#         keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
#         keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
#     ]
# )

In [42]:
# def generate_text(model, start_text, token_to_id, id_to_token, max_len=100, temperature=1.0):
#     # Encode start text
#     token_ids, attention_mask = tokenize_and_build_token_id(
#         token_to_id, [start_text], max_seq_len=512
#     )
    
#     generated = list(token_ids[0].numpy())  # flatten out
#     mask = list(attention_mask[0].numpy())
    
#     for _ in range(max_len):
#         # Trim to last 512 tokens
#         x_tokens = np.array([generated[-512:]])
#         x_mask   = np.array([mask[-512:]])

#         # Forward pass with both inputs
#         logits = model((x_tokens, x_mask), training=False)

#         # Take last position logits
#         next_logits = logits[0, -1] / temperature
#         probs = tf.nn.softmax(next_logits).numpy()

#         # Sample next token
#         next_id = np.random.choice(len(probs), p=probs)

#         # Append
#         generated.append(next_id)
#         mask.append(1)  # mark as valid token

#     # Decode
#     return ''.join(id_to_token[i] for i in generated if i in id_to_token)


In [43]:
id_to_token_dict = {v: k for k, v in token_to_id_dict.items()}

In [44]:
# start = "Akshat Khatri"
# result = generate_text(model, start, token_to_id_dict, id_to_token_dict, max_len=100, temperature=0.8)
# print(result)

In [45]:
# from datasets import load_dataset
# dataset = load_dataset('wikitext', 'wikitext-103-v1')# Concatenate train + validation + test
# all_texts = []
# for split in ["train", "validation", "test"]:
#     all_texts.extend(dataset[split]["text"])

# # Remove empty lines
# all_texts = [t.strip() for t in all_texts if t.strip() != ""]

# # Join into one giant string
# big_text = "\n".join(all_texts)

# # Write to file
# with open("wikitext_full.txt", "w", encoding="utf-8") as f:
#     f.write(big_text)

# print("Saved dataset to wikitext_full.txt")


In [46]:
# inputs, targets, masks = prepare_book_training_data(
#     book_text, 
#     token_to_id_dict, 
#     context_length=512
# )

# print(f"Created {len(inputs)} training examples")
# print(f"Input shape: {inputs.shape}")
# print(f"Target shape: {targets.shape}")

In [47]:
# import numpy as np

# # Assuming you already have:
# # - prepare_book_training_data()
# # - token_to_id_dict

# file_path = r"/home/akshat/GPT_from_scratch/text_data/wikitext_full.txt"

# inputs_list, targets_list, masks_list = [], [], []

# buffer = ""
# with open(file_path, "r", encoding="utf-8") as f:
#     for line in f:
#         buffer += line.strip() + " "
#         # Process every ~5000 chars to avoid memory spike
#         if len(buffer) > 5000:
#             inp, tgt, msk = prepare_book_training_data(
#                 buffer, token_to_id_dict, context_length=512
#             )
#             inputs_list.append(inp)
#             targets_list.append(tgt)
#             masks_list.append(msk)
#             buffer = ""  # reset buffer

# # Process any leftover buffer
# if buffer.strip():
#     inp, tgt, msk = prepare_book_training_data(
#         buffer, token_to_id_dict, context_length=512
#     )
#     inputs_list.append(inp)
#     targets_list.append(tgt)
#     masks_list.append(msk)

# # Concatenate all batches into final arrays
# inputs = np.concatenate(inputs_list, axis=0)
# targets = np.concatenate(targets_list, axis=0)
# masks = np.concatenate(masks_list, axis=0)

# print(f"Created {len(inputs)} training examples")
# print(f"Input shape: {inputs.shape}")
# print(f"Target shape: {targets.shape}")
# print(f"Masks shape: {masks.shape}")

In [48]:
import tensorflow as tf
import numpy as np
from typing import Dict, List
import os
from tqdm import tqdm


def create_tf_example(input_ids: List[int], target_ids: List[int], attention_mask: List[int]) -> bytes:
    """
    Create a serialized TFRecord example from input-target pair.
    
    Args:
        input_ids: Token sequence for model input
        target_ids: Token sequence for model targets (shifted by 1)
        attention_mask: Mask for valid tokens (1) vs padding (0)
        
    Returns:
        Serialized TFRecord example
    """
    feature = {
        'input_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=input_ids)),
        'target_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=target_ids)),
        'attention_mask': tf.train.Feature(int64_list=tf.train.Int64List(value=attention_mask))
    }
    
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()


def convert_text_to_tfrecord(
    text_file_path: str,
    token_to_id_dict: Dict[str, int],
    output_dir: str,
    context_length: int = 512,
    records_per_file: int = 1000,
    pad_value: int = 0
) -> str:
    """
    Convert text file to TFRecord files for GPT training.
    
    Process:
    1. Read and tokenize entire text file
    2. Create sliding windows of context_length + 1 tokens
    3. Split each window into input/target pairs (shifted by 1)
    4. Save as TFRecord files with specified number of records per file
    
    Args:
        text_file_path: Path to your text file (e.g., WikiText-103)
        token_to_id_dict: Character-to-ID mapping dictionary
        output_dir: Directory to save TFRecord files
        context_length: Sequence length for training
        records_per_file: Number of examples per TFRecord file
        pad_value: Token ID used for unknown characters
        
    Returns:
        Path to output directory containing TFRecord files
    """
    # Setup
    os.makedirs(output_dir, exist_ok=True)
    print(f"Reading text file: {text_file_path}")
    
    # Step 1: Load and tokenize text
    with open(text_file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    print(f"Text length: {len(text):,} characters")
    
    # Convert each character to token ID
    print("Tokenizing text...")
    token_ids = [token_to_id_dict.get(char, pad_value) for char in text]
    print(f"Token length: {len(token_ids):,} tokens")
    
    # Step 2: Calculate output size
    num_examples = (len(token_ids) - context_length) // context_length
    print(f"Will create {num_examples:,} training examples")
    
    # Step 3: Process sliding windows and write TFRecord files
    file_count = 0
    examples_in_current_file = 0
    writer = None
    
    print("Creating TFRecord files...")
    
    # Slide window across token sequence
    for i in tqdm(range(0, len(token_ids) - context_length, context_length)):
        
        # Extract window of tokens
        window = token_ids[i:i + context_length + 1]
        if len(window) < context_length + 1:
            break
            
        # Create input-target pair (GPT training format)
        input_ids = window[:-1]    # First 512 tokens: [t1, t2, ..., t512]
        target_ids = window[1:]    # Shifted by 1: [t2, t3, ..., t513]
        attention_mask = [1] * context_length  # All valid tokens (no padding)
        
        # Start new TFRecord file if needed
        if writer is None or examples_in_current_file >= records_per_file:
            if writer is not None:
                writer.close()
            
            tfrecord_filename = os.path.join(output_dir, f'train_{file_count:04d}.tfrecord')
            writer = tf.io.TFRecordWriter(tfrecord_filename)
            file_count += 1
            examples_in_current_file = 0
        
        # Write training example to current TFRecord file
        tf_example = create_tf_example(input_ids, target_ids, attention_mask)
        writer.write(tf_example)
        examples_in_current_file += 1
    
    # Cleanup
    if writer is not None:
        writer.close()
    
    # Step 4: Save summary information
    print(f"\nConversion complete!")
    print(f"Created {file_count} TFRecord files in: {output_dir}")
    print(f"Total examples: {num_examples:,}")
    
    # Write metadata file
    metadata = {
        'context_length': context_length,
        'vocab_size': len(token_to_id_dict),
        'num_examples': num_examples,
        'num_files': file_count,
        'records_per_file': records_per_file
    }
    
    metadata_path = os.path.join(output_dir, 'metadata.txt')
    with open(metadata_path, 'w') as f:
        for key, value in metadata.items():
            f.write(f"{key}: {value}\n")
    
    print(f"Metadata saved to: {metadata_path}")
    
    return output_dir


def create_tf_data_pipeline(
    tfrecord_dir: str,
    context_length: int = 512,
    batch_size: int = 32,
    shuffle_buffer: int = 1000,
    prefetch_buffer: int = tf.data.AUTOTUNE
) -> tf.data.Dataset:
    """
    Create tf.data pipeline from TFRecord files for training.
    
    Process:
    1. Find all TFRecord files in directory
    2. Create dataset that reads and parses TFRecord examples
    3. Apply shuffling, batching, and prefetching for efficient training
    
    Args:
        tfrecord_dir: Directory containing TFRecord files
        context_length: Expected sequence length in records
        batch_size: Number of examples per training batch
        shuffle_buffer: Size of shuffle buffer (larger = more random)
        prefetch_buffer: Number of batches to prefetch (AUTOTUNE = automatic)
        
    Returns:
        tf.data.Dataset ready for model.fit()
    """
    # Step 1: Find TFRecord files
    tfrecord_files = tf.io.gfile.glob(os.path.join(tfrecord_dir, "*.tfrecord"))
    print(f"Found {len(tfrecord_files)} TFRecord files")
    
    # Step 2: Define how to parse each TFRecord example
    feature_description = {
        'input_ids': tf.io.FixedLenFeature([context_length], tf.int64),
        'target_ids': tf.io.FixedLenFeature([context_length], tf.int64),
        'attention_mask': tf.io.FixedLenFeature([context_length], tf.int64)
    }
    
    def parse_tfrecord_example(example_proto):
        """Parse a single TFRecord example into model inputs and targets."""
        # Parse the serialized example
        parsed_features = tf.io.parse_single_example(example_proto, feature_description)
        
        # Convert to correct data types
        input_ids = tf.cast(parsed_features['input_ids'], tf.int32)
        target_ids = tf.cast(parsed_features['target_ids'], tf.int32)
        attention_mask = tf.cast(parsed_features['attention_mask'], tf.int32)
        
        # Return in format expected by your GPT model: ((input_ids, attention_mask), targets)
        model_inputs = (input_ids, attention_mask)
        model_targets = target_ids
        
        return model_inputs, model_targets
    
    # Step 3: Create and configure dataset pipeline
    dataset = tf.data.TFRecordDataset(tfrecord_files)
    dataset = dataset.map(parse_tfrecord_example, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(prefetch_buffer)
    
    return dataset


# Example usage (commented out)
if __name__ == "__main__":
    """
    Example usage for WikiText-103 or similar large text files
    """
    
    # # Step 1: Define your vocabulary (replace with actual token_to_id_dict)
    # example_vocab = your_token_to_id_dict
    
    # # Step 2: Convert text to TFRecord format (run once)
    # tfrecord_dir = convert_text_to_tfrecord(
    #     text_file_path=r'/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
    #     token_to_id_dict=example_vocab,
    #     output_dir='./tfrecords',
    #     context_length=512,
    #     records_per_file=1000
    # )
    
    # # Step 3: Create training pipeline (use for training)
    # train_dataset = create_tf_data_pipeline(
    #     tfrecord_dir=tfrecord_dir,
    #     context_length=512,
    #     batch_size=16
    # )
    
    # print("TFRecord pipeline ready for training!")
    # # Now you can use train_dataset with your model's .fit() method
    
    pass

In [49]:
# # Step 1: Define your vocabulary (replace with actual token_to_id_dict)
# example_vocab = token_to_id_dict

# # Step 2: Convert text to TFRecord format (run once)
# tfrecord_dir = convert_text_to_tfrecord(
#     text_file_path=r'/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
#     token_to_id_dict=example_vocab,
#     output_dir='./tfrecords',
#     context_length=128,
#     records_per_file=1000
# )

In [50]:
# Step 3: Create training and validation pipeline (use for training)
def create_train_val_datasets(tfrecord_dir: str, 
                             context_length: int,
                             batch_size: int = 32,
                             val_split: float = 0.1):
    """
    Create training and validation datasets from TFRecord files
    """
    # Find all TFRecord files
    tfrecord_files = tf.io.gfile.glob(os.path.join(tfrecord_dir, "*.tfrecord"))
    print(f"Found {len(tfrecord_files)} TFRecord files")
    
    # Split files for train/val
    num_val_files = max(1, int(len(tfrecord_files) * val_split))
    val_files = tfrecord_files[:num_val_files]
    train_files = tfrecord_files[num_val_files:]
    
    print(f"Using {len(train_files)} files for training, {len(val_files)} for validation")
    
    # Feature description
    feature_description = {
        'input_ids': tf.io.FixedLenFeature([context_length], tf.int64),
        'target_ids': tf.io.FixedLenFeature([context_length], tf.int64),
        'attention_mask': tf.io.FixedLenFeature([context_length], tf.int64)
    }
    
    def parse_function(example_proto):
        parsed_features = tf.io.parse_single_example(example_proto, feature_description)
        
        input_ids = tf.cast(parsed_features['input_ids'], tf.int32)
        target_ids = tf.cast(parsed_features['target_ids'], tf.int32)
        attention_mask = tf.cast(parsed_features['attention_mask'], tf.int32)
        
        # Return in format your model expects: ([input_ids, attention_mask], targets)
        return (input_ids, attention_mask), target_ids
    
    # Create training dataset
    train_dataset = tf.data.TFRecordDataset(train_files)
    train_dataset = train_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    train_dataset = train_dataset.shuffle(5000)  # Shuffle buffer
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    
    # Create validation dataset
    val_dataset = tf.data.TFRecordDataset(val_files)
    val_dataset = val_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    val_dataset = val_dataset.batch(batch_size)
    val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)
    
    return train_dataset, val_dataset


In [51]:
# from typing import List
# import os
# from tqdm import tqdm


# def create_tf_example(input_ids: List[int], target_ids: List[int], attention_mask: List[int]) -> bytes:
#     """
#     Create a serialized TFRecord example from input-target pair.
    
#     Args:
#         input_ids: Token sequence for model input
#         target_ids: Token sequence for model targets (shifted by 1)
#         attention_mask: Mask for valid tokens (1) vs padding (0)
        
#     Returns:
#         Serialized TFRecord example
#     """
#     feature = {
#         'input_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=input_ids)),
#         'target_ids': tf.train.Feature(int64_list=tf.train.Int64List(value=target_ids)),
#         'attention_mask': tf.train.Feature(int64_list=tf.train.Int64List(value=attention_mask))
#     }
    
#     example = tf.train.Example(features=tf.train.Features(feature=feature))
#     return example.SerializeToString()



# from tqdm import tqdm

# def convert_text_to_tfrecord_sp(
#     text_file_path: str,
#     sp_model_path: str,
#     output_dir: str,
#     context_length: int = 512,
#     records_per_file: int = 1000,
#     overlap_size: int = 64,
#     chunk_size: int = 100_000  # Number of characters read at a time
# ) -> str:
#     """
#     Streaming version of text to TFRecord conversion with SentencePiece.
#     Reads and tokenizes file incrementally to limit memory usage.
#     """
#     os.makedirs(output_dir, exist_ok=True)
#     file_size = os.path.getsize(text_file_path)
#     print(f"Reading and tokenizing text in chunks from: {text_file_path}")
    
#     # Load SentencePiece processor
#     sp = spm.SentencePieceProcessor()
#     sp.load(sp_model_path)
    
#     print(f"Loaded SentencePiece model from: {sp_model_path}")
#     print(f"Vocabulary size: {sp.get_piece_size()}")

#     buffer_tokens = []
#     step_size = context_length - overlap_size
#     file_count = 0
#     examples_in_current_file = 0
#     writer = None
    
#     with open(text_file_path, 'r', encoding='utf-8') as file:
#         with tqdm(total=file_size, unit='B', unit_scale=True, desc='Processing text') as pbar:
#             while True:
#                 chunk = file.read(chunk_size)
#                 if not chunk:
#                     break
#                 buffer_tokens.extend(sp.encode_as_ids(chunk))
#                 pbar.update(len(chunk.encode('utf-8')))  # update by bytes read
                
#                 # Process windows to create examples
#                 while len(buffer_tokens) >= context_length + 1:
#                     window = buffer_tokens[:context_length + 1]
#                     input_ids = window[:-1]
#                     target_ids = window[1:]
#                     attention_mask = [1] * context_length
                    
#                     if writer is None or examples_in_current_file >= records_per_file:
#                         if writer:
#                             writer.close()
#                         tfrecord_path = os.path.join(output_dir, f'train_{file_count:04d}.tfrecord')
#                         writer = tf.io.TFRecordWriter(tfrecord_path)
#                         file_count += 1
#                         examples_in_current_file = 0
                    
#                     tf_example = create_tf_example(input_ids, target_ids, attention_mask)
#                     writer.write(tf_example)
#                     examples_in_current_file += 1
                    
#                     # Slide window forward by step_size tokens
#                     buffer_tokens = buffer_tokens[step_size:]

#     # Process any remaining tokens
#     while len(buffer_tokens) >= context_length + 1:
#         window = buffer_tokens[:context_length + 1]
#         input_ids = window[:-1]
#         target_ids = window[1:]
#         attention_mask = [1] * context_length
        
#         if writer is None or examples_in_current_file >= records_per_file:
#             if writer:
#                 writer.close()
#             tfrecord_path = os.path.join(output_dir, f'train_{file_count:04d}.tfrecord')
#             writer = tf.io.TFRecordWriter(tfrecord_path)
#             file_count += 1
#             examples_in_current_file = 0

#         tf_example = create_tf_example(input_ids, target_ids, attention_mask)
#         writer.write(tf_example)
#         examples_in_current_file += 1
#         buffer_tokens = buffer_tokens[step_size:]

#     if writer:
#         writer.close()

#     num_examples = file_count * records_per_file  # Approximate count

#     print(f"\nConversion complete!")
#     print(f"Created {file_count} TFRecord files in: {output_dir}")
#     print(f"Approximate total examples: {num_examples}")

#     metadata = {
#         'context_length': context_length,
#         'vocab_size': sp.get_piece_size(),
#         'num_examples': num_examples,
#         'num_files': file_count,
#         'records_per_file': records_per_file,
#         'overlap_size': overlap_size,
#         'sp_model_path': sp_model_path,
#         'tokenization': 'SentencePiece'
#     }

#     metadata_path = os.path.join(output_dir, 'metadata.txt')
#     with open(metadata_path, 'w') as f:
#         for key, value in metadata.items():
#             f.write(f"{key}: {value}\n")
#     print(f"Metadata saved to: {metadata_path}")

#     return output_dir



# def create_tf_data_pipeline_sp(
#     tfrecord_dir: str,
#     context_length: int = 512,
#     batch_size: int = 32,
#     shuffle_buffer: int = 1000,
#     prefetch_buffer: int = tf.data.AUTOTUNE
# ) -> tf.data.Dataset:
#     """
#     Create tf.data pipeline from TFRecord files for training (SentencePiece version).
    
#     Process:
#     1. Find all TFRecord files in directory
#     2. Create dataset that reads and parses TFRecord examples
#     3. Apply shuffling, batching, and prefetching for efficient training
    
#     Args:
#         tfrecord_dir: Directory containing TFRecord files
#         context_length: Expected sequence length in records
#         batch_size: Number of examples per training batch
#         shuffle_buffer: Size of shuffle buffer (larger = more random)
#         prefetch_buffer: Number of batches to prefetch (AUTOTUNE = automatic)
        
#     Returns:
#         tf.data.Dataset ready for model.fit()
#     """
#     # Step 1: Find TFRecord files
#     tfrecord_files = tf.io.gfile.glob(os.path.join(tfrecord_dir, "*.tfrecord"))
#     print(f"Found {len(tfrecord_files)} TFRecord files")
    
#     if not tfrecord_files:
#         raise FileNotFoundError(f"No TFRecord files found in {tfrecord_dir}")
    
#     # Step 2: Define how to parse each TFRecord example
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([context_length], tf.int64),
#         'target_ids': tf.io.FixedLenFeature([context_length], tf.int64),
#         'attention_mask': tf.io.FixedLenFeature([context_length], tf.int64)
#     }
    
#     def parse_tfrecord_example(example_proto):
#         """Parse a single TFRecord example into model inputs and targets."""
#         # Parse the serialized example
#         parsed_features = tf.io.parse_single_example(example_proto, feature_description)
        
#         # Convert to correct data types
#         input_ids = tf.cast(parsed_features['input_ids'], tf.int32)
#         target_ids = tf.cast(parsed_features['target_ids'], tf.int32)
#         attention_mask = tf.cast(parsed_features['attention_mask'], tf.int32)
        
#         # Return in format expected by your GPT model: ((input_ids, attention_mask), targets)
#         model_inputs = (input_ids, attention_mask)
#         model_targets = target_ids
        
#         return model_inputs, model_targets
    
#     # Step 3: Create and configure dataset pipeline
#     dataset = tf.data.TFRecordDataset(tfrecord_files)
#     dataset = dataset.map(parse_tfrecord_example, num_parallel_calls=tf.data.AUTOTUNE)
#     dataset = dataset.shuffle(shuffle_buffer)
#     dataset = dataset.batch(batch_size)
#     dataset = dataset.prefetch(prefetch_buffer)
    
#     return dataset


# # Example usage
# # if __name__ == "__main__":
# #     """
# #     Example usage assuming you have a pre-trained SentencePiece model
# #     """
    
# #     # Your pre-trained SentencePiece model path
# #     sp_model_path = '/home/akshat/GPT_from_scratch/notebooks/spm_gpt.model'  # You provide this
    
# #     # Step 1: Convert text to TFRecords using your pre-trained model
# #     tfrecord_dir = convert_text_to_tfrecord_sp(
# #         text_file_path=r'/home/akshat/GPT_from_scratch/text_data/BookCorpus3_cleaned.txt',
# #         sp_model_path=sp_model_path,  # Your trained model
# #         output_dir='./tfrecords',
# #         context_length=CONTEXT_LEN,  # Match your CONTEXT_LEN
# #         records_per_file=1000,
# #         overlap_size = CONTEXT_LEN // 2,
# #         chunk_size = 150000
# #     )
    
# #     # # Step 2: Create training pipeline
# #     # train_dataset = create_tf_data_pipeline_sp(
# #     #     tfrecord_dir=tfrecord_dir,
# #     #     context_length=128,  # Match your CONTEXT_LEN
# #     #     batch_size=16
# #     # )
    
# #     print("SentencePiece TFRecord pipeline ready for training!")
    
# #     # Step 3: Load your SentencePiece model for vocab size and generation
# #     sp = spm.SentencePieceProcessor()
# #     sp.load(sp_model_path)
# #     VOCAB_SIZE = sp.get_piece_size()
    
# #     print(f"Vocabulary size: {VOCAB_SIZE}")
    
#     # Now you can use:
#     # - sp for tokenization in generation
#     # - train_dataset for model.fit()
#     # - VOCAB_SIZE for your model definition


In [52]:
# # Create DATASETS
# train_dataset, val_dataset = create_train_val_datasets(
#     tfrecord_dir='./tfrecords',
#     context_length=CONTEXT_LEN,
#     batch_size=16,
#     val_split=0.1
# )


# print("TFRecord pipeline ready for training!")

In [53]:
# import shutil
# import os

# # Remove existing tfrecords directory
# if os.path.exists('./tfrecords'):
#     shutil.rmtree('./tfrecords')
#     print("Removed existing tfrecords directory")

In [54]:
# # SOLUTION 1: Recreate TFRecords with correct context length
# # Delete the existing tfrecords directory and recreate with CONTEXT_LEN


# # Recreate with correct context length
# tfrecord_dir = convert_text_to_tfrecord(
#     text_file_path=r'/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
#     token_to_id_dict=token_to_id_dict,  # Make sure this variable is defined
#     output_dir='./tfrecords',
#     context_length=CONTEXT_LEN,  # Use your actual context length (128)
#     records_per_file=1000
# )

# # Now create datasets with matching context length
# train_dataset, val_dataset = create_train_val_datasets(
#     tfrecord_dir='./tfrecords',
#     context_length=CONTEXT_LEN,  # This will now match
#     batch_size=16,
#     val_split=0.1
# )

# # Calculate steps per epoch
# records_per_file = 1000  
# tfrecord_files = tf.io.gfile.glob(os.path.join("./tfrecords", "*.tfrecord"))
# total_examples = len(tfrecord_files) * records_per_file
# train_examples = int(total_examples * 0.9)
# steps_per_epoch = train_examples // 16

# print(f"Files: {len(tfrecord_files)}")
# print(f"Total examples: {total_examples}")
# print(f"Steps per epoch: {steps_per_epoch}") # When combined with batch_size sees the whole data

In [55]:
import os
import tensorflow as tf

def prepare_tfrecords(
    text_file_path: str,
    token_to_id_dict: dict,
    context_length: int = 128,
    records_per_file: int = 1000,
    output_base_dir: str = './tfrecords',
    version_name: str = None,
    batch_size: int = 16,
    val_split: float = 0.1
):
    """
    Create TFRecords from text and return train/val datasets.
    Stores TFRecords in a versioned folder to avoid overwriting previous ones.

    Args:
        text_file_path: Path to input text file.
        token_to_id_dict: Character-to-id dictionary.
        context_length: Sequence length for training.
        records_per_file: Number of examples per TFRecord file.
        output_base_dir: Base folder to store TFRecords.
        version_name: Optional unique folder name. If None, uses context_length.
        batch_size: Batch size for dataset.
        val_split: Fraction of data to use as validation.

    Returns:
        train_dataset, val_dataset, steps_per_epoch
    """

    # Determine output folder
    if version_name is None:
        version_name = f"context_{context_length}_bs{batch_size}"
    output_dir = os.path.join(output_base_dir, version_name)
    os.makedirs(output_dir, exist_ok=True)

    # Convert text to TFRecords
    tfrecord_dir = convert_text_to_tfrecord(
        text_file_path=text_file_path,
        token_to_id_dict=token_to_id_dict,
        output_dir=output_dir,
        context_length=context_length,
        records_per_file=records_per_file
    )

    # Create train/val datasets
    train_dataset, val_dataset = create_train_val_datasets(
        tfrecord_dir=tfrecord_dir,
        context_length=context_length,
        batch_size=batch_size,
        val_split=val_split
    )
    def count_tfrecord_examples(tfrecord_files):
        count = 0
        for tfrecord_file in tfrecord_files:
            for _ in tf.data.TFRecordDataset(tfrecord_file):
                count += 1
        return count

    # inside prepare_tfrecords
    tfrecord_files = tf.io.gfile.glob(os.path.join(tfrecord_dir, "*.tfrecord"))
    total_examples = count_tfrecord_examples(tfrecord_files)
    train_examples = int(total_examples * (1 - val_split))
    steps_per_epoch = train_examples // batch_size

    print(f"TFRecord folder: {tfrecord_dir}")
    print(f"Total examples: {total_examples}")
    print(f"Train examples: {train_examples}")
    print(f"Steps per epoch: {steps_per_epoch}")

    return train_dataset, val_dataset, steps_per_epoch


In [56]:
# train_dataset,val_dataset,steps_per_epoch = prepare_tfrecords(
#     text_file_path='/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
#     token_to_id_dict=token_to_id_dict,
#     context_length=128,
#     batch_size=16
# )

In [57]:
# train_ds_24, val_ds_24, steps_24 = prepare_tfrecords(
#     text_file_path='/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
#     token_to_id_dict=token_to_id_dict,
#     context_length=128,
#     batch_size=24
# )

In [58]:
# train_ds_32, val_ds_32, steps_32 = prepare_tfrecords(
#     text_file_path='/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
#     token_to_id_dict=token_to_id_dict,
#     context_length=128,
#     batch_size=32
# )

In [59]:
# train_ds_44, val_ds_44, steps_44 = prepare_tfrecords(
#     text_file_path='/home/akshat/GPT_from_scratch/notebooks/wikitext_full.txt',
#     token_to_id_dict=token_to_id_dict,
#     context_length=128,
#     batch_size=44
# )

In [60]:
train_ds_64, val_ds_64, steps_64 = prepare_tfrecords(
    text_file_path='/home/akshat/GPT_from_scratch/text_data/jane_austen_clean.txt',
    token_to_id_dict=token_to_id_dict,
    context_length=CONTEXT_LEN,
    batch_size=64,
)

train_ds_64 = train_ds_64.shuffle(10000)
val_ds_64 = val_ds_64.shuffle(10000)

Reading text file: /home/akshat/GPT_from_scratch/text_data/jane_austen_clean.txt
Text length: 4,347,531 characters
Tokenizing text...
Token length: 4,347,531 tokens
Will create 67,929 training examples
Creating TFRecord files...


100%|██████████| 67930/67930 [00:01<00:00, 38916.76it/s]



Conversion complete!
Created 68 TFRecord files in: ./tfrecords/context_64_bs64
Total examples: 67,929
Metadata saved to: ./tfrecords/context_64_bs64/metadata.txt
Found 68 TFRecord files
Using 62 files for training, 6 for validation


2025-08-30 08:24:27.780313: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:390] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-08-30 08:24:27.855721: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-30 08:24:27.945970: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-30 08:24:28.133403: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-30 08:24:28.509933: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-30 08:24:29.307072: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-08-30 08:24:30.943169: I tensorflow/core/framework/local_rendezvous.cc:407] Lo

TFRecord folder: ./tfrecords/context_64_bs64
Total examples: 67930
Train examples: 61137
Steps per epoch: 955


In [61]:
# DECODER_BLOCKS = 5
# ATTENTION_HEADS = 8

# GPT_model = GPT(D_MODEL,VOCAB_SIZE,CONTEXT_LEN,ATTENTION_HEADS,0.00001,DECODER_BLOCKS,0.3)
# _ = GPT_model((token_ids, attention_mask))
# GPT_model.summary(expand_nested=True)

# # training (stable): use logits
# loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# opt = keras.optimizers.AdamW(learning_rate=lr_schedule, weight_decay=1e-4, clipnorm=1.0)
# GPT_model.compile(optimizer=opt, loss=loss) # type: ignore

# # inference probs (when you actually need them)
# logits = GPT_model((token_ids, attention_mask), training=False)
# probs = keras.ops.softmax(logits, axis=-1)

In [62]:
EPOCHS = 100
DECODER_BLOCKS = 6
ATTENTION_HEADS = 4
DROPOUT_RATE = 0.2
STEPS_PER_EPOCH = steps_64  # e.g. 955
TOTAL_STEPS = EPOCHS * STEPS_PER_EPOCH  # 95,500
WARMUP_STEPS = int(0.1 * TOTAL_STEPS)   # 9,550
LR = 1e-5

lr_schedule = CosineDecayWithWarmup(
    warmup_steps=WARMUP_STEPS,
    total_steps=TOTAL_STEPS,
    peak_learning_rate=5e-4,
    min_learning_rate=5e-6
)

model = GPT(D_MODEL,VOCAB_SIZE,CONTEXT_LEN,ATTENTION_HEADS,0.00001,DECODER_BLOCKS,DROPOUT_RATE)
# Compile your model (same as before)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = keras.optimizers.AdamW(learning_rate=lr_schedule, weight_decay=1e-4, clipnorm=1.0,beta_2=0.95) # type: ignore
model.compile(optimizer=opt, loss=loss,metrics=['accuracy']) # type: ignore

In [ ]:
callbacks = [
    # Save model every epoch with epoch number
    keras.callbacks.ModelCheckpoint(
        filepath='/home/akshat/GPT_from_scratch/notebooks/word_level_checkpoints/model_epoch_{epoch:02d}_val_loss_{val_loss:.4f}.keras',
        save_freq='epoch',
        save_best_only=False,
        verbose=1
    ),
    
    # Save best model separately
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    
    # More reasonable early stopping
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,  # Wait 10 epochs before stopping
        restore_best_weights=True,
        verbose=1
    ),

    keras.callbacks.CSVLogger('training_log.csv'),
    keras.callbacks.TensorBoard(
        log_dir="./word_logs", 
        histogram_freq=1, 
        profile_batch=0,
        write_graph=True
    )
]

# Training
print("Starting training...")
print(f"Total epochs: ",EPOCHS)
print(f"Steps per epoch: {STEPS_PER_EPOCH}")
print(f"Total steps: {EPOCHS * STEPS_PER_EPOCH}")

history = model.fit(
    train_ds_64,
    validation_data=val_ds_64,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks,
)

print("Training completed!")
print(f"Best validation loss: {min(history.history['val_loss']):.4f}")

# Plot training curves
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
if 'learning_rate' in history.history:
    plt.plot(history.history['learning_rate'])
    plt.title('Learning Rate Schedule')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')

plt.subplot(1, 3, 3)
train_perplexity = [np.exp(loss) for loss in history.history['loss']]
val_perplexity = [np.exp(loss) for loss in history.history['val_loss']]
plt.plot(train_perplexity, label='Train Perplexity')
plt.plot(val_perplexity, label='Val Perplexity')
plt.title('Perplexity')
plt.xlabel('Epoch')
plt.ylabel('Perplexity')
plt.legend()
plt.tight_layout()
plt.show()

Starting training...
Total epochs:  100
Steps per epoch: 955
Total steps: 95500
Epoch 1/100


2025-08-30 08:25:27.744229: I external/local_xla/xla/service/service.cc:163] XLA service 0x15667fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-30 08:25:27.744262: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-08-30 08:25:28.264570: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-30 08:25:29.314684: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:62] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. gpt_1_1/decoder_block_0_1/self_attention_layer_8_1/dropout/random_uniform/RandomUniform
2025-08-30 08:25:29.571210: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator compile_loss/sparse_cat

327/955 ━━━━━━━━━━━━━━━━━━━━ 3:28 333ms/step - accuracy: 0.2684 - loss: 5.7694

2025-08-30 08:27:53.823627: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2025-08-30 08:27:59.161874: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_123', 4 bytes spill stores, 4 bytes spill loads

2025-08-30 08:27:59.346130: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_112', 8 bytes spill stores, 8 bytes spill loads

2025-08-30 08:27:59.978910: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_18107', 20 bytes spill stores, 20 bytes spill loads

2025-08-30 08:28:00.569359: I external/local_xla/xla/stream_executor

955/955 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.3446 - loss: 3.9893

...


Epoch 50: saving model to /home/akshat/GPT_from_scratch/notebooks/word_level_checkpoints/model_epoch_50_val_loss_1.2032.keras


2025-08-30 11:09:55.896736: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10633177541358495391
2025-08-30 11:09:55.896812: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 18257925123837337519
2025-08-30 11:09:55.896860: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 9461568589138220430



Epoch 50: val_loss improved from 1.20376 to 1.20318, saving model to best_model.keras
955/955 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.6217 - loss: 1.1707 - val_accuracy: 0.6152 - val_loss: 1.2032
Epoch 51/100
955/955 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - accuracy: 0.6186 - loss: 1.1746
Epoch 51: saving model to /home/akshat/GPT_from_scratch/notebooks/word_level_checkpoints/model_epoch_51_val_loss_1.1951.keras

Epoch 51: val_loss improved from 1.20318 to 1.19508, saving model to best_model.keras
955/955 ━━━━━━━━━━━━━━━━━━━━ 345s 360ms/step - accuracy: 0.6196 - loss: 1.1713 - val_accuracy: 0.6186 - val_loss: 1.1951
Epoch 52/100
 14/955 ━━━━━━━━━━━━━━━━━━━━ 6:58 445ms/step - accuracy: 0.6215 - loss: 1.1607

2025-08-30 11:16:18.098477: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10633177541358495391



Epoch 52: saving model to /home/akshat/GPT_from_scratch/notebooks/word_level_checkpoints/model_epoch_52_val_loss_1.1959.keras


In [303]:
# # Now training should work
# history = model.fit(
#     train_ds_32,
#     validation_data=val_ds_32,
#     epochs=50,
#     steps_per_epoch=steps_32,
#     callbacks=[
#         keras.callbacks.ModelCheckpoint(filepath='model_epoch_{epoch:02d}.keras',save_freq='epoch'),  # saves with epoch number   save_freq='epoch',                        # save every epoch    save_best_only=False,                     # save all epochs    verbose=1)
#         keras.callbacks.EarlyStopping(patience=1, restore_best_weights=True, verbose=1),
#         keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=10, verbose=1),
#         keras.callbacks.CSVLogger('training_log.csv'),
#         keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1, profile_batch=0)
#     ]
# )

In [304]:
# pick a small dummy batch
import tensorflow as tf

dum_model = GPT(D_MODEL, VOCAB_SIZE, CONTEXT_LEN, 8, 0.00001, 4, 0.1)
dummy_input = tf.constant([[0] * CONTEXT_LEN], dtype=tf.int32)  # batch_size=1, length=CONTEXT_LEN
dum_model.save('model_epoch_1.keras')
# run the embeddings layer only
pos_layer = dum_model.get_layer('init_embeddings')  # or however your layer is named
try:
    pos_emb = pos_layer(dummy_input)
    print("✅ Positional embeddings are working. Shape:", pos_emb.shape)
except Exception as e:
    print("❌ Embedding test failed:", e)


✅ Positional embeddings are working. Shape: (1, 128, 64)


/home/akshat/ml/ml-venv/lib/python3.12/site-packages/keras/src/saving/saving_api.py:107: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  return saving_lib.save_model(model, filepath)


In [305]:
# Step 4: Save the final model
model.save('final_gpt_model.keras')
print("Model saved as 'final_gpt_model.keras'")

# Optional: Save training history
import pickle
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("Training history saved as 'training_history.pkl'")

# Step 5: Load model later (when needed)
def load_trained_model():
    """Load your saved model"""
    loaded_model = keras.models.load_model('best_model.keras')  # or 'final_gpt_model.keras'
    return loaded_model

# Usage for inference later:
# model = load_trained_model()

Model saved as 'final_gpt_model.keras'
Training history saved as 'training_history.pkl'


In [306]:
GPT_model.save('dum_GPT.keras')

In [307]:
import gradio as gr
import numpy as np
import tensorflow as tf
import keras

# token_to_id_dict = tokenize_and_build_vocabulary_tf([r'/home/akshat/GPT_from_scratch/text_data/jane_austen_clean.txt'])
id_to_token_dict = {id_val: token for token, id_val in token_to_id_dict.items()}

# Use the latest and best model - try the best_model.keras first, then latest checkpoint
try:
    model = keras.models.load_model(r'/home/akshat/GPT_from_scratch/notebooks/char_level_checkpoints/model_epoch_299_val_loss_0.0130.keras')
    print("✅ Loaded best_model.keras")
except:
    try:
        model = keras.models.load_model(r'/home/akshat/GPT_from_scratch/notebooks/char_level_checkpoints/model_epoch_163_val_loss_0.0303.keras')
        print("✅ Loaded epoch 163 model")
    except:
        model = keras.models.load_model(r'/home/akshat/GPT_from_scratch/notebooks/char_level_checkpoints/model_epoch_161_val_loss_0.0305.keras')
        print("✅ Loaded epoch 161 model")

CONTEXT_LEN = model._context_length  # Use the model's actual context length

# Debug: Print vocabulary info
print(f"Vocabulary size: {len(token_to_id_dict)}")
print(f"Model type: {type(model)}")

# Get model info from your custom GPT model
try:
    print(f"Model vocab size: {model._vocab_size}")
    print(f"Model context length: {model._context_length}")
    print(f"Model d_model: {model._d_model}")
    print(f"Model attention heads: {model._attention_heads}")
    print(f"Model decoder blocks: {model._decoder_blocks}")
    print(f"Vocab size matches model: {model._vocab_size == len(token_to_id_dict)}")
    
    if model._vocab_size != len(token_to_id_dict):
        print(f"⚠️  VOCAB SIZE MISMATCH! Model expects {model._vocab_size}, got {len(token_to_id_dict)}")
    
except Exception as e:
    print(f"Error getting model info: {e}")

print(f"Sample characters in vocab: {list(token_to_id_dict.keys())[:30]}")
print(f"Common characters present: {['a' in token_to_id_dict, 'e' in token_to_id_dict, ' ' in token_to_id_dict, '.' in token_to_id_dict]}")

# Check for problematic characters in the gibberish output
gibberish = "4ff.mtm 64m86rfstmfm?.fmmftms777mtmkf  tm7n7m77m77"
print(f"Checking gibberish characters:")
for char in set(gibberish):
    if char in token_to_id_dict:
        print(f"  '{char}' -> ID {token_to_id_dict[char]} ✓")
    else:
        print(f"  '{char}' -> NOT IN VOCAB ✗")

def encode_text(text, token_to_id_dict):
    """Encode text to token IDs using character-level tokenizer - convert to lowercase since dataset is lowercase"""
    # Convert input to lowercase since your dataset was lowercased
    text = text.lower()
    
    token_ids = []
    for char in text:
        if char in token_to_id_dict:
            token_ids.append(token_to_id_dict[char])
        else:
            print(f"Warning: '{char}' (ord: {ord(char)}) not in vocabulary, skipping")
            continue
    return token_ids

def decode_ids(token_ids, id_to_token_dict):
    """Decode token IDs back to text using character-level tokenizer"""
    text = ""
    for token_id in token_ids:
        if token_id in id_to_token_dict:
            text += id_to_token_dict[token_id]
        else:
            print(f"Warning: token ID {token_id} not in vocabulary")
    return text

def get_special_token_ids():
    """Get special token IDs - adjust these based on your tokenizer setup"""
    # For Jane Austen data, likely no special PAD token, use newline as EOS
    pad_id = token_to_id_dict.get('<PAD>', None)
    eos_id = token_to_id_dict.get('\n', None)  # Use newline as natural stopping point
    print(f"Special tokens - PAD: {pad_id}, EOS (newline): {eos_id}")
    return pad_id, eos_id

def top_k_sampling(logits, k=10):
    """Sample from logits using top-k sampling"""
    # Ensure we don't sample more than available tokens
    k = min(k, len(logits))
    
    values, indices = tf.math.top_k(logits, k=k)
    last_val = values[-1]
    filtered_logits = tf.where(
        logits < last_val,
        tf.fill(tf.shape(logits), float('-inf')),
        logits
    )
    probs = tf.nn.softmax(filtered_logits).numpy()
    
    # Add small epsilon to avoid numerical issues
    probs = probs + 1e-10
    probs = probs / np.sum(probs)
    
    return np.random.choice(len(probs), p=probs)

def generate_response(prompt, max_length=100, temperature=0.7, top_k=10, use_argmax=False):
    if not prompt.strip():
        return ""
    
    print(f"\n--- Generation Debug ---")
    print(f"Input prompt: '{prompt}' (will be lowercased)")
    
    # Tokenize prompt with character-level tokenizer
    input_tokens = encode_text(prompt, token_to_id_dict)
    print(f"Input tokens: {input_tokens}")
    print(f"Input tokens decoded back: '{decode_ids(input_tokens, id_to_token_dict)}'")
    
    if not input_tokens:
        return "Error: Could not tokenize input"
    
    # Truncate if longer than context length
    if len(input_tokens) > CONTEXT_LEN:
        input_tokens = input_tokens[-CONTEXT_LEN:]
    
    generated_tokens = input_tokens.copy()
    pad_id, eos_id = get_special_token_ids()
    
    print(f"Starting generation with {len(input_tokens)} input tokens...")
    
    for step in range(max_length):
        # Prepare inputs - pad from left to maintain most recent context
        input_ids = np.zeros((1, CONTEXT_LEN), dtype=np.int32)
        attention_mask = np.zeros((1, CONTEXT_LEN), dtype=np.int32)
        
        # Place tokens at the end of the context window
        current_len = min(len(generated_tokens), CONTEXT_LEN)
        start_idx = CONTEXT_LEN - current_len
        input_ids[0, start_idx:] = generated_tokens[-current_len:]
        attention_mask[0, start_idx:] = 1
        
        # Model forward pass
        try:
            logits = model((input_ids, attention_mask), training=False)
            next_token_logits = logits[0, -1, :]
            
            # Apply temperature
            if not use_argmax:
                next_token_logits = next_token_logits / temperature
        except Exception as e:
            print(f"Model forward pass error: {e}")
            break
        
        # Sample next token
        try:
            if use_argmax:
                # Use argmax (greedy) sampling for testing
                next_token = int(np.argmax(next_token_logits))
            else:
                # Use top-k sampling
                next_token = top_k_sampling(next_token_logits, k=top_k)
        except Exception as e:
            print(f"Sampling error: {e}")
            break
        
        # Debug: Print first few tokens
        if step < 10:
            sampled_char = id_to_token_dict.get(next_token, f"<UNK:{next_token}>")
            prob = float(tf.nn.softmax(next_token_logits)[next_token])
            print(f"Step {step}: Token {next_token} -> '{sampled_char}' (prob: {prob:.4f})")
        
        # Check if token is valid
        if next_token >= len(id_to_token_dict):
            print(f"Warning: Invalid token {next_token}, vocab size is {len(id_to_token_dict)}")
            break
        
        # Stop on special tokens
        if pad_id is not None and next_token == pad_id:
            print(f"Stopping at step {step}: hit PAD token")
            break
        if eos_id is not None and next_token == eos_id and step > 10:  # Don't stop too early
            print(f"Stopping at step {step}: hit EOS token (newline)")
            break
        
        generated_tokens.append(int(next_token))
        
        # Maintain sliding window
        if len(generated_tokens) > CONTEXT_LEN:
            generated_tokens = generated_tokens[-CONTEXT_LEN:]
    
    # Decode only the newly generated tokens
    new_tokens = generated_tokens[len(input_tokens):]
    response = decode_ids(new_tokens, id_to_token_dict)
    print(f"Generated {len(new_tokens)} new tokens: {new_tokens[:20]}...")  # Show first 20
    print(f"Generated response: '{response}'")
    print(f"--- End Debug ---\n")
    
    return response.strip()

def chat_fn(message, history, temperature, max_length, top_k, use_argmax):
    if not message.strip():
        return "", history
    
    bot_response = generate_response(message, max_length=max_length, temperature=temperature, top_k=top_k, use_argmax=use_argmax)
    history.append((message, bot_response))
    return "", history

# Quick test with the better model
print("Testing improved model:")
test_cases = ["the", "elizabeth", "it is a"]

for prompt in test_cases:
    print(f"\nTesting with: '{prompt}'")
    tokens = encode_text(prompt, token_to_id_dict)
    
    # Create model input
    input_ids = np.zeros((1, 256), dtype=np.int32)
    attention_mask = np.zeros((1, 256), dtype=np.int32)
    input_ids[0, -len(tokens):] = tokens
    attention_mask[0, -len(tokens):] = 1
    
    # Get model predictions
    logits = model((input_ids, attention_mask), training=False)
    next_token_logits = logits[0, -1, :]
    
    # Show top 5 predictions
    top_probs, top_indices = tf.nn.top_k(tf.nn.softmax(next_token_logits), k=5)
    print("Top 5 predictions:")
    for i in range(5):
        token_id = int(top_indices[i])
        prob = float(top_probs[i])
        char = id_to_token_dict.get(token_id, f"UNK_{token_id}")
        print(f"  {i+1}. '{char}' (ID: {token_id}) - {prob:.4f}")

# Test with longer context
print(f"\nTesting with longer Jane Austen context:")
long_prompt = "it is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a"
tokens = encode_text(long_prompt, token_to_id_dict)
print(f"Context: '{long_prompt}'")
print(f"Context length: {len(tokens)} tokens")

# Use reasonable context length
context_tokens = tokens[-100:] if len(tokens) > 100 else tokens

input_ids = np.zeros((1, 256), dtype=np.int32)
attention_mask = np.zeros((1, 256), dtype=np.int32)
input_ids[0, -len(context_tokens):] = context_tokens
attention_mask[0, -len(context_tokens):] = 1

logits = model((input_ids, attention_mask), training=False)
next_token_logits = logits[0, -1, :]

top_probs, top_indices = tf.nn.top_k(tf.nn.softmax(next_token_logits), k=5)
print("Top 5 predictions after long context:")
for i in range(5):
    token_id = int(top_indices[i])
    prob = float(top_probs[i])
    char = id_to_token_dict.get(token_id, f"UNK_{token_id}")
    print(f"  {i+1}. '{char}' (ID: {token_id}) - {prob:.4f}")

with gr.Blocks(title="My Character-Level GPT Bot Trained in Tensorflow", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 Chat with Akshat's Character-Level GPT Model")
    gr.Markdown("Ask me anything! I'm a GPT model trained from scratch with character-level tokenization on Jane Austen data. Be gentle with me :)")
    
    # Add vocab info
    gr.Markdown(f"**Model Info:** Vocabulary size: {len(token_to_id_dict)} characters")
    
    chatbot = gr.Chatbot(label="Conversation", height=400, show_copy_button=True)
    
    with gr.Row():
        msg = gr.Textbox(label="Your message", placeholder="Type your message here...", scale=4)
        send_btn = gr.Button("Send", scale=1, variant="primary")
    
    with gr.Row():
        temperature = gr.Slider(minimum=0.1, maximum=1.5, value=0.3, step=0.05, label="Temperature")
        max_length = gr.Slider(minimum=10, maximum=200, value=30, step=10, label="Max Length")
        top_k = gr.Slider(minimum=1, maximum=50, value=5, step=1, label="Top-K Sampling")
        use_argmax = gr.Checkbox(label="Use Argmax (Greedy) - for testing", value=True)
    
    clear_btn = gr.Button("Clear Chat", variant="secondary")
    
    # Event handlers
    msg.submit(chat_fn, [msg, chatbot, temperature, max_length, top_k, use_argmax], [msg, chatbot])
    send_btn.click(chat_fn, [msg, chatbot, temperature, max_length, top_k, use_argmax], [msg, chatbot])
    clear_btn.click(lambda: [], None, chatbot)

if __name__ == "__main__":
    demo.launch(
        share=True,          # Generate public share link
        server_name="127.0.0.1",
        server_port=6019,
        show_error=True
    )

ValueError: File not found: filepath=/home/akshat/GPT_from_scratch/notebooks/char_level_checkpoints/model_epoch_161_val_loss_0.0305.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
def generate_text(prompt, max_length=50, temperature=1.0):
    input_tokens = [token_to_id_dict.get(c, 0) for c in prompt if c in token_to_id_dict]
    if len(input_tokens) == 0:
        input_tokens = [0]

    if len(input_tokens) > CONTEXT_LEN:
        input_tokens = input_tokens[-CONTEXT_LEN:]

    input_ids = np.zeros((1, CONTEXT_LEN), dtype=np.int32)
    input_ids[0, -len(input_tokens):] = input_tokens

    attention_mask = np.zeros((1, CONTEXT_LEN), dtype=np.int32)
    attention_mask[0, -len(input_tokens):] = 1

    generated_tokens = input_tokens.copy()

    for _ in range(max_length):
        predictions = model.predict([input_ids, attention_mask], verbose=0)
        next_token_logits = predictions[0, -1, :] / temperature
        probabilities = tf.nn.softmax(next_token_logits).numpy()
        next_token = np.random.choice(len(probabilities), p=probabilities)

        if next_token == 0:
            break

        generated_tokens.append(next_token)

        # Update input_ids and attention_mask
        if len(generated_tokens) > CONTEXT_LEN:
            generated_tokens = generated_tokens[-CONTEXT_LEN:]

        input_ids[0, -len(generated_tokens):] = generated_tokens
        attention_mask[0, -len(generated_tokens):] = 1

    id_to_token = {v: k for k, v in token_to_id_dict.items()}
    return ''.join([id_to_token.get(t, '') for t in generated_tokens[len(input_tokens):]]).strip()


# Simple console loop
while True:
    prompt = input("You: ")
    if prompt.lower() in ["quit", "exit"]:
        break
    response = generate_text(prompt, max_length=50, temperature=0.8)
    print("GPT: ", response)

GPT:  VxxZEtRh5HHeYXZtxnhx5ICxxetRx5(ZxZMqx55xejhPxYW4Rx
GPT:  RcT2-LeJPeRHVpMKhzo37xBoxti
GPT:  qt-45HxGefn5ZZx48.UxMeTRuOLzRHxt
GPT:  RxHxtX5RmeMxHd2tZ


KeyboardInterrupt: Interrupted by user

In [ ]:
token_to_id_dict.keys()

dict_keys(['\n', ' ', '!', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])

In [ ]:
id_to_token_dict.values()

dict_values(['\n', ' ', '!', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])

In [ ]:
print(sinusoidal_lookup_table.shape)  # should be (CONTEXT_LEN, D_MODEL)

(128, 128)


In [ ]:
import tensorflow as tf
import keras
import numpy as np

# Model configuration
CONTEXT_LEN = 128
D_MODEL = 128
VOCAB_SIZE = 94

def test_sinusoidal_lookup_table():
    """Test the sinusoidal lookup table creation"""
    print("🔍 Testing sinusoidal lookup table creation...")
    try:
        sinusoidal_lookup_table = prepare_sinusoidal_lookup_table(D_MODEL, CONTEXT_LEN)
        print(f"✅ Sinusoidal lookup table created successfully. Shape: {sinusoidal_lookup_table.shape}")
        print(f"   Table type: {type(sinusoidal_lookup_table)}")
        print(f"   Table dtype: {sinusoidal_lookup_table.dtype}")
        return sinusoidal_lookup_table
    except Exception as e:
        print(f"❌ Sinusoidal lookup table creation failed: {e}")
        return None

def test_positional_embeddings(sinusoidal_lookup_table):
    """Test the positional embeddings layer"""
    print("\n🔍 Testing InitializePositionalEmbeddings layer...")
    
    try:
        # Create the layer
        pos_layer = InitializePositionalEmbeddings(
            d_model=D_MODEL,
            vocab_size=VOCAB_SIZE,
            sinusoidal_lookup_table=sinusoidal_lookup_table,
            name="test_pos_embeddings"
        )
        
        # Test input
        dummy_input = tf.constant([[1, 2, 3, 4, 5] + [0] * (CONTEXT_LEN - 5)], dtype=tf.int32)
        print(f"   Input shape: {dummy_input.shape}")
        
        # Forward pass
        pos_emb = pos_layer(dummy_input)
        print(f"✅ Positional embeddings working. Output shape: {pos_emb.shape}")
        print(f"   Expected shape: (1, {CONTEXT_LEN}, {D_MODEL})")
        
        # Check if embeddings are reasonable
        print(f"   Output range: [{tf.reduce_min(pos_emb):.4f}, {tf.reduce_max(pos_emb):.4f}]")
        print(f"   Output mean: {tf.reduce_mean(pos_emb):.4f}")
        
        return pos_layer, pos_emb
        
    except Exception as e:
        print(f"❌ Positional embeddings test failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def test_layer_normalization():
    """Test LayerNormalization layer"""
    print("\n🔍 Testing LayerNormalization layer...")
    
    try:
        ln = LayerNormalization(eps=1e-5, name="test_ln")
        test_input = tf.random.normal((2, CONTEXT_LEN, D_MODEL))
        
        output = ln(test_input)
        print(f"✅ LayerNormalization working. Input shape: {test_input.shape}, Output shape: {output.shape}")
        
        # Check normalization properties
        mean = tf.reduce_mean(output, axis=-1)
        var = tf.reduce_mean(tf.square(output - tf.expand_dims(mean, -1)), axis=-1)
        print(f"   Output mean (should be ~0): {tf.reduce_mean(mean):.6f}")
        print(f"   Output variance (should be ~1): {tf.reduce_mean(var):.6f}")
        
        return ln, output
        
    except Exception as e:
        print(f"❌ LayerNormalization test failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def test_your_self_attention():
    """Test YOUR custom SelfAttentionLayer"""
    print("\n🔍 Testing YOUR SelfAttentionLayer...")
    
    try:
        # Create your attention layer with correct parameter name
        attn_layer = SelfAttentionLayer(attention_heads=8, name="test_attention")
        
        # Prepare test inputs
        batch_size = 2
        seq_len = CONTEXT_LEN
        embeddings = tf.random.normal((batch_size, seq_len, D_MODEL))
        
        # Create attention mask (1 for real tokens, 0 for padding)
        attention_mask = tf.ones((batch_size, seq_len), dtype=tf.float32)
        # Make some positions masked (set to 0)
        attention_mask = attention_mask.numpy()
        attention_mask[0, 50:] = 0  # Mask second half of first sequence
        attention_mask[1, 80:] = 0  # Mask last part of second sequence
        attention_mask = tf.constant(attention_mask)
        
        print(f"   Input embeddings shape: {embeddings.shape}")
        print(f"   Attention mask shape: {attention_mask.shape}")
        print(f"   Mask sample - seq 1: {attention_mask[0, :10].numpy()} ... {attention_mask[0, -10:].numpy()}")
        
        # Test with your layer's expected input format: (embeddings, mask)
        output = attn_layer([embeddings, attention_mask])
        print(f"✅ SelfAttentionLayer working. Output shape: {output.shape}")
        
        # Check output properties
        print(f"   Output range: [{tf.reduce_min(output):.4f}, {tf.reduce_max(output):.4f}]")
        print(f"   Output mean: {tf.reduce_mean(output):.4f}")
        print(f"   Output std: {tf.math.reduce_std(output):.4f}")
        
        # Verify attention heads are working
        print(f"   Attention heads: {attn_layer.attention_heads}")
        print(f"   d_head: {attn_layer.d_head}")
        print(f"   d_model: {attn_layer.d_model}")
        
        return attn_layer, output
        
    except Exception as e:
        print(f"❌ SelfAttentionLayer test failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def test_your_decoder_block():
    """Test YOUR custom DecoderBlock"""
    print("\n🔍 Testing YOUR DecoderBlock...")
    
    try:
        # Create your decoder block
        decoder = DecoderBlock(
            d_model=D_MODEL,
            n_heads=8,
            dropout_rate=0.1,
            epsilon=1e-5,
            name="test_decoder"
        )
        
        # Prepare test inputs
        batch_size = 2
        seq_len = CONTEXT_LEN
        test_input = tf.random.normal((batch_size, seq_len, D_MODEL))
        attention_mask = tf.ones((batch_size, seq_len), dtype=tf.float32)
        
        # Make some positions masked
        attention_mask = attention_mask.numpy()
        attention_mask[0, 60:] = 0
        attention_mask[1, 90:] = 0
        attention_mask = tf.constant(attention_mask)
        
        print(f"   Input shape: {test_input.shape}")
        print(f"   Attention mask shape: {attention_mask.shape}")
        
        # Test training=False
        output = decoder(test_input, attention_mask, training=False)
        print(f"✅ DecoderBlock working (training=False). Output shape: {output.shape}")
        
        # Test training=True
        output_train = decoder(test_input, attention_mask, training=True)
        print(f"✅ DecoderBlock working (training=True). Output shape: {output_train.shape}")
        
        # Check residual connections work (output should be different from input)
        input_mean = tf.reduce_mean(test_input)
        output_mean = tf.reduce_mean(output)
        print(f"   Input mean: {input_mean:.4f}, Output mean: {output_mean:.4f}")
        print(f"   Output range: [{tf.reduce_min(output):.4f}, {tf.reduce_max(output):.4f}]")
        
        return decoder, output
        
    except Exception as e:
        print(f"❌ DecoderBlock test failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def test_your_full_model(sinusoidal_lookup_table):
    """Test YOUR complete GPT model"""
    print("\n🔍 Testing YOUR complete GPT model...")
    
    try:
        # Create model exactly as you do
        model = GPT(
            d_model=D_MODEL,
            vocab_size=VOCAB_SIZE,
            context_length=CONTEXT_LEN,
            attention_heads=8,
            epsilon=1e-5,
            decoder_blocks=4,
            dropout_rate=0.1,
            sinusoidal_lookup_table=sinusoidal_lookup_table,
            name="test_gpt"
        )
        
        # Prepare test inputs
        token_ids = tf.constant([[1, 2, 3, 4, 5] + [0] * (CONTEXT_LEN - 5)], dtype=tf.int32)
        attention_mask = tf.ones((1, CONTEXT_LEN), dtype=tf.float32)
        
        # Set mask to 0 for padding tokens
        attention_mask = attention_mask.numpy()
        attention_mask[0, 5:] = 0  # Only first 5 tokens are real
        attention_mask = tf.constant(attention_mask)
        
        print(f"   Token IDs shape: {token_ids.shape}")
        print(f"   Token IDs sample: {token_ids[0, :10].numpy()}")
        print(f"   Attention mask shape: {attention_mask.shape}")
        print(f"   Mask sample: {attention_mask[0, :10].numpy()}")
        
        # Test forward pass
        logits = model([token_ids, attention_mask], training=False)
        print(f"✅ Full model forward pass successful!")
        print(f"   Output logits shape: {logits.shape}")
        print(f"   Expected shape: (1, {CONTEXT_LEN}, {VOCAB_SIZE})")
        
        # Check output properties
        print(f"   Logits range: [{tf.reduce_min(logits):.4f}, {tf.reduce_max(logits):.4f}]")
        print(f"   Logits mean: {tf.reduce_mean(logits):.4f}")
        
        # Test with different batch size
        token_ids_batch = tf.constant([
            [1, 2, 3] + [0] * (CONTEXT_LEN - 3),
            [4, 5, 6, 7] + [0] * (CONTEXT_LEN - 4)
        ], dtype=tf.int32)
        attention_mask_batch = tf.ones((2, CONTEXT_LEN), dtype=tf.float32)
        attention_mask_batch = attention_mask_batch.numpy()
        attention_mask_batch[0, 3:] = 0  # First seq has 3 real tokens
        attention_mask_batch[1, 4:] = 0  # Second seq has 4 real tokens
        attention_mask_batch = tf.constant(attention_mask_batch)
        
        logits_batch = model([token_ids_batch, attention_mask_batch], training=False)
        print(f"✅ Batch processing successful! Output shape: {logits_batch.shape}")
        
        # Test training mode
        logits_train = model([token_ids, attention_mask], training=True)
        print(f"✅ Training mode successful! Output shape: {logits_train.shape}")
        
        return model, logits
        
    except Exception as e:
        print(f"❌ Full model test failed: {e}")
        import traceback
        traceback.print_exc()
        return None, None

def test_specific_embedding_issue():
    """Test the specific embedding issue you encountered"""
    print("\n🔍 Testing the specific embedding issue from your original code...")
    
    try:
        # Create model exactly as you did
        sinusoidal_lookup_table = prepare_sinusoidal_lookup_table(D_MODEL, CONTEXT_LEN)
        dum_model = GPT(D_MODEL, VOCAB_SIZE, CONTEXT_LEN, 8, 0.00001, 4, 0.1, sinusoidal_lookup_table)
        
        # Test exactly as you did
        dummy_input = tf.constant([[0] * CONTEXT_LEN], dtype=tf.int32)
        
        # Get the embeddings layer
        pos_layer = dum_model.get_layer('init_embeddings')
        
        # Run the embeddings layer
        pos_emb = pos_layer(dummy_input)
        print(f"✅ Your original embedding test now works! Shape: {pos_emb.shape}")
        print(f"   Input was all zeros: {dummy_input[0, :5].numpy()}")
        print(f"   Output range: [{tf.reduce_min(pos_emb):.4f}, {tf.reduce_max(pos_emb):.4f}]")
        
        return True
        
    except Exception as e:
        print(f"❌ Original embedding test still fails: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_model_compilation_and_training(model):
    """Test model compilation and training capability"""
    print("\n🔍 Testing model compilation and training...")
    
    try:
        # Compile model
        model.compile(
            optimizer=keras.optimizers.AdamW(learning_rate=1e-4),
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        print("✅ Model compilation successful!")
        
        # Create dummy training data
        batch_size = 4
        dummy_x = tf.random.uniform((batch_size, CONTEXT_LEN), maxval=VOCAB_SIZE, dtype=tf.int32)
        dummy_mask = tf.ones((batch_size, CONTEXT_LEN), dtype=tf.float32)
        # Create some realistic masking
        for i in range(batch_size):
            seq_len = tf.random.uniform([], minval=10, maxval=CONTEXT_LEN, dtype=tf.int32)
            dummy_mask = dummy_mask.numpy()
            dummy_mask[i, seq_len:] = 0
            dummy_mask = tf.constant(dummy_mask)
        
        dummy_y = tf.random.uniform((batch_size, CONTEXT_LEN), maxval=VOCAB_SIZE, dtype=tf.int32)
        
        print(f"   Training data shapes: X={dummy_x.shape}, mask={dummy_mask.shape}, Y={dummy_y.shape}")
        
        # Test prediction
        predictions = model.predict([dummy_x, dummy_mask], verbose=0)
        print(f"✅ Model prediction successful! Predictions shape: {predictions.shape}")
        
        # Test training step
        loss = model.train_on_batch([dummy_x, dummy_mask], dummy_y)
        print(f"✅ Training step successful! Loss: {loss}")
        
        # Test model summary
        print(f"\n📊 Model has {model.count_params():,} parameters")
        
        return True
        
    except Exception as e:
        print(f"❌ Model compilation/training test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

def run_all_tests_for_your_model():
    """Run all tests specifically for your model implementation"""
    print("🚀 Starting comprehensive testing for YOUR custom model implementation...\n")
    
    # Test 1: Sinusoidal lookup table
    sinusoidal_lookup_table = test_sinusoidal_lookup_table()
    if sinusoidal_lookup_table is None:
        print("❌ Cannot proceed without sinusoidal lookup table")
        return
    
    # Test 2: Positional embeddings
    pos_layer, pos_emb = test_positional_embeddings(sinusoidal_lookup_table)
    
    # Test 3: Layer normalization
    ln_layer, ln_output = test_layer_normalization()
    
    # Test 4: Your self attention
    attn_layer, attn_output = test_your_self_attention()
    
    # Test 5: Your decoder block
    decoder_layer, decoder_output = test_your_decoder_block()
    
    # Test 6: Your full model
    model, logits = test_your_full_model(sinusoidal_lookup_table)
    
    # Test 7: Your specific embedding issue
    embedding_issue_fixed = test_specific_embedding_issue()
    
    # Test 8: Model compilation and training
    compilation_success = False
    if model is not None:
        compilation_success = test_model_compilation_and_training(model)
    
    # Summary
    print("\n" + "="*70)
    print("🎯 TESTING SUMMARY FOR YOUR CUSTOM MODEL:")
    print("="*70)
    
    tests = [
        ("Sinusoidal Lookup Table", sinusoidal_lookup_table is not None),
        ("Positional Embeddings", pos_emb is not None),
        ("Layer Normalization", ln_output is not None),
        ("YOUR Self Attention Layer", attn_output is not None),
        ("YOUR Decoder Block", decoder_output is not None),
        ("YOUR Full Model Forward Pass", logits is not None),
        ("YOUR Original Embedding Issue", embedding_issue_fixed),
        ("Model Compilation & Training", compilation_success)
    ]
    
    passed = sum(1 for _, result in tests if result)
    total = len(tests)
    
    for test_name, result in tests:
        status = "✅ PASSED" if result else "❌ FAILED"
        print(f"{test_name:.<50} {status}")
    
    print(f"\n🏆 Overall: {passed}/{total} tests passed")
    
    if passed == total:
        print("🎉 All tests passed! Your model is working perfectly!")
        print("🚀 Your model is ready for training and inference!")
    elif passed >= total - 2:
        print("🎊 Almost all tests passed! Your model is mostly working correctly!")
        print("🔧 Check the failed tests above for minor issues.")
    else:
        print("⚠️  Some tests failed. Please fix the issues before training.")
    
    return model if logits is not None else None

# Run the tests
if __name__ == "__main__":
    final_model = run_all_tests_for_your_model()
    
    if final_model is not None:
        print(f"\n🎁 Model returned successfully!")
        print(f"   Total parameters: {final_model.count_params():,}")
        print(f"   Ready for: training, inference, and saving!")

🚀 Starting comprehensive testing for YOUR custom model implementation...

🔍 Testing sinusoidal lookup table creation...
✅ Sinusoidal lookup table created successfully. Shape: (128, 128)
   Table type: <class 'tensorflow.python.framework.ops.EagerTensor'>
   Table dtype: <dtype: 'float32'>

🔍 Testing InitializePositionalEmbeddings layer...
❌ Positional embeddings test failed: Unrecognized keyword arguments passed to InitializePositionalEmbeddings: {'sinusoidal_lookup_table': <tf.Tensor: shape=(128, 128), dtype=float32, numpy=
array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
         1.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [ 8.4147096e-01,  5.4030228e-01,  7.6172042e-01, ...,
         1.0000000e+00,  1.1547820e-04,  1.0000000e+00],
       [ 9.0929741e-01, -4.1614684e-01,  9.8704624e-01, ...,
         9.9999994e-01,  2.3095640e-04,  1.0000000e+00],
       ...,
       [-6.1604047e-01,  7.8771454e-01,  9.9029869e-01, ...,
         9.9986106e-01,  1.4434273e-02,  9

Traceback (most recent call last):
  File "/tmp/ipykernel_206177/3617421218.py", line 29, in test_positional_embeddings
    pos_layer = InitializePositionalEmbeddings(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_206177/3965528913.py", line 11, in __init__
    super().__init__(**kwargs)
  File "/home/akshat/ml/ml-venv/lib/python3.12/site-packages/keras/src/layers/layer.py", line 291, in __init__
    raise ValueError(
ValueError: Unrecognized keyword arguments passed to InitializePositionalEmbeddings: {'sinusoidal_lookup_table': <tf.Tensor: shape=(128, 128), dtype=float32, numpy=
array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
         1.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [ 8.4147096e-01,  5.4030228e-01,  7.6172042e-01, ...,
         1.0000000e+00,  1.1547820e-04,  1.0000000e+00],
       [ 9.0929741e-01, -4.1614684e-01,  9.8704624e-01, ...,
         9.9999994e-01,  2.3095640e-04,  1.0000000e+00],
       ...,
       [-6.1604047e-01

   Input shape: (2, 128, 128)
   Attention mask shape: (2, 128)
✅ DecoderBlock working (training=False). Output shape: (2, 128, 128)
✅ DecoderBlock working (training=True). Output shape: (2, 128, 128)
   Input mean: 0.0003, Output mean: -0.0041
   Output range: [-4.3376, 4.7553]

🔍 Testing YOUR complete GPT model...
❌ Full model test failed: Unrecognized keyword arguments passed to GPT: {'sinusoidal_lookup_table': <tf.Tensor: shape=(128, 128), dtype=float32, numpy=
array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
         1.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [ 8.4147096e-01,  5.4030228e-01,  7.6172042e-01, ...,
         1.0000000e+00,  1.1547820e-04,  1.0000000e+00],
       [ 9.0929741e-01, -4.1614684e-01,  9.8704624e-01, ...,
         9.9999994e-01,  2.3095640e-04,  1.0000000e+00],
       ...,
       [-6.1604047e-01,  7.8771454e-01,  9.9029869e-01, ...,
         9.9986106e-01,  1.4434273e-02,  9.9989581e-01],
       [ 3.2999083e-01,  9.4398415e-01,  7.474

Traceback (most recent call last):
  File "/tmp/ipykernel_206177/3617421218.py", line 186, in test_your_full_model
    model = GPT(
            ^^^^
  File "/tmp/ipykernel_206177/1791215164.py", line 64, in __init__
    super().__init__(**kwargs)
  File "/home/akshat/ml/ml-venv/lib/python3.12/site-packages/keras/src/models/model.py", line 158, in __init__
    Layer.__init__(self, *args, **kwargs)
  File "/home/akshat/ml/ml-venv/lib/python3.12/site-packages/keras/src/layers/layer.py", line 291, in __init__
    raise ValueError(
ValueError: Unrecognized keyword arguments passed to GPT: {'sinusoidal_lookup_table': <tf.Tensor: shape=(128, 128), dtype=float32, numpy=
array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
         1.0000000e+00,  0.0000000e+00,  1.0000000e+00],
       [ 8.4147096e-01,  5.4030228e-01,  7.6172042e-01, ...,
         1.0000000e+00,  1.1547820e-04,  1.0000000e+00],
       [ 9.0929741e-01, -4.1614684e-01,  9.8704624e-01, ...,
         9.9999994e-01,  2.30956

In [ ]:
def generate_response_from_bot(model, token_to_id_dict, prompt, max_length=100, temperature=0.7, context_len=128):
    """Generate text response from your GPT model"""
    
    # Tokenize input
    input_tokens = [token_to_id_dict.get(char, 0) for char in prompt]
    
    # Handle context length
    if len(input_tokens) > context_len:
        input_tokens = input_tokens[-context_len:]
    
    # Pad to context length
    input_ids = np.zeros(context_len, dtype=np.int32)
    if len(input_tokens) > 0:
        input_ids[-len(input_tokens):] = input_tokens
    
    # Create attention mask
    attention_mask = np.zeros(context_len, dtype=np.int32)
    if len(input_tokens) > 0:
        attention_mask[-len(input_tokens):] = 1
    
    # Prepare for model
    input_ids = np.expand_dims(input_ids, axis=0)
    attention_mask = np.expand_dims(attention_mask, axis=0)
    
    # Generate response token by token
    generated_tokens = input_tokens.copy()
    id_to_token = {v: k for k, v in token_to_id_dict.items()}
    
    for _ in range(max_length):
        # Get model predictions
        predictions = model.predict([input_ids, attention_mask], verbose=0)
        
        # Get last token logits (find the last non-zero position in attention mask)
        last_pos = np.sum(attention_mask[0]) - 1
        if last_pos < 0:
            last_pos = 0
        next_token_logits = predictions[0, last_pos, :] / temperature
        
        # Convert to probabilities
        probabilities = tf.nn.softmax(next_token_logits).numpy()
        
        # Sample next token
        next_token = np.random.choice(len(probabilities), p=probabilities)
        
        # Stop if we hit a stop token or newline
        if next_token == 0 or (next_token in token_to_id_dict.values() and id_to_token[next_token] == '\n'):
            break
            
        generated_tokens.append(next_token)
        
        # Update input for next iteration
        if len(generated_tokens) > context_len:
            generated_tokens = generated_tokens[-context_len:]
        
        # Create new input
        new_input_ids = np.zeros((1, context_len), dtype=np.int32)
        if len(generated_tokens) > 0:
            new_input_ids[0, -len(generated_tokens):] = generated_tokens
        
        new_attention_mask = np.zeros((1, context_len), dtype=np.int32)
        if len(generated_tokens) > 0:
            new_attention_mask[0, -len(generated_tokens):] = 1
        
        input_ids = new_input_ids
        attention_mask = new_attention_mask
    
    # Convert tokens back to text
    response_tokens = generated_tokens[len(input_tokens):]  # Only the new tokens
    response = ''.join([id_to_token.get(token, '') for token in response_tokens])
    
    return response.strip()

In [ ]:
generate_response_from_bot(model,token_to_id_dict,prompt = 'yoyo')

"]8'Vi3A;#sFöXöxr3ö][xöM6!dlwx—$pb:Orxx1JkW0:pöyyö;94œ!ööGHQöG:‘::$fwrg3Rg!R!/gxrgg/PöJIYPlö6öJ%6RpLp"